# T cell annotation

In [ ]:
import anndata
import numpy as np
import pandas as pd
#from matplotlib_venn import venn2
from glob import glob

import scanpy as sc
import anndata as ad
import anndata
import pickle
import matplotlib.pyplot as plt

from tqdm import tqdm
import gc
import tarfile
#import scirpy as ir
import collections
from glob import glob
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import anndata
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import tarfile
#import community.community_louvain as community_louvain
import networkx as nx
from sklearn.neighbors import kneighbors_graph
from scipy import stats
import seaborn as sns
import collections
import random
from scipy.stats import zscore
#from kneed import KneeLocator
random.seed(10)

In [ ]:
#!conda env list

In [ ]:
#import scvi

In [ ]:
#!pip install matplotlib_venn
#!pip install kneed

### Load in the data

In [ ]:
tcell_ann_concat = anndata.read_h5ad('/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_sc.h5ad')

In [ ]:
tcell_ann_concat

In [ ]:
table = pd.pivot_table(tcell_ann_concat.obs,index ='PatientID',aggfunc='count') 

In [ ]:
table

In [ ]:
Sample_list = ["R096_Tc17_up","R096_Tc17_low","RH01","RH02","RH03","RH04","RH05","RH06","RH07","RH08","RH09","RH10","RH11","RH12","RH13","RH14","UA01","UA02","UA03","UA04","UA05","UA06","UA07","UA08","UA09","UA10","UA11","UA12",'UA13','UA14','UA15','UA19','UA20','UA21','UA22','UA23','UA24','UA26','UA30','UA31','UA32','UA33','UA34','UA35','UA36','UA37','UA38','UA39','UA40','UA41']

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden', "CD3D","CD3E","CD3G", 'CD4', 'CD8A','CD8B','NCAM1','TRAC','TRBC1','TRBC2','TRDC','TRGC1','TRGC2','CD1D'])

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
sc.tl.leiden(tcell_ann_concat,resolution=0.5,key_added='leiden_0.5')

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'])

In [ ]:
tcell_ann_concat.obs['CD4_Pos']=tcell_ann_concat.to_df()['CD4']>0

In [ ]:
tcell_ann_concat.obs['CD8_Pos']=tcell_ann_concat.to_df()['CD8A']>0

In [ ]:
tcell_ann_concat

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden','leiden_0.5',"CD3D","CD3E","CD3G", 'CD4', 'CD8A','GVHD_Grade_1'])

In [ ]:
DoublePos = tcell_ann_concat[tcell_ann_concat.obs['CD4_Pos']== True]

In [ ]:
DoublePos = DoublePos[DoublePos.obs['CD8_Pos']== True]

In [ ]:
DoublePos

In [ ]:
sc.pl.umap(DoublePos, color=['leiden',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
tcell_ann_concat.obs

In [ ]:
for patient in tcell_ann_concat.obs['PatientID'].unique():
    tcell_ann_concat.obs['color'] = 'Else'
    tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID'].isin([patient]),'color'] = tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID'].isin([patient]),'Days_post-Transplant']
    meta ='color'
    randomize=np.arange(len(tcell_ann_concat.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=tcell_ann_concat.obsm["X_umap"][:,0][randomize],y=tcell_ann_concat.obsm["X_umap"][:,1][randomize], hue=tcell_ann_concat.obs[meta][randomize],s=3).set(title=patient)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
tcell_ann_concat

In [ ]:
sc.tl.leiden(tcell_ann_concat,resolution=0.1, key_added='leiden_48')#resolution=0.1,


In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_48'])

In [ ]:
new_cluster_names = ['CD8','CD4']
tcell_ann_concat.rename_categories('leiden_48', new_cluster_names)

sc.pl.umap(tcell_ann_concat, color='leiden_48', legend_loc='on data', title='', frameon=False, save='.pdf')

In [ ]:
cd4_ann = tcell_ann_concat[tcell_ann_concat.obs["leiden_48"].isin(['CD4'])]
cd8_ann = tcell_ann_concat[tcell_ann_concat.obs["leiden_48"].isin(['CD8'])]

sc.pl.umap(cd4_ann, color=['leiden_48',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

sc.pl.umap(cd8_ann, color=['leiden_48',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
cd4_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/cd4_ann.h5ad'
cd4_ann.write(cd4_ann_loc)

cd8_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/cd8_ann.h5ad'
cd8_ann.write(cd8_ann_loc)

In [ ]:
tcell_ann_concat.obs["LPorIE"]

In [ ]:
tcell_ann_IE = tcell_ann_concat[tcell_ann_concat.obs["LPorIE"]== 'Intraepithelial Cells']
tcell_ann_LP =tcell_ann_concat[tcell_ann_concat.obs["LPorIE"]== 'Lamina Propria Cells']

In [ ]:
#sc.pp.neighbors(tcell_ann_IE, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)
#sc.tl.leiden(tcell_ann_IE,resolution=0.1)#resolution=0.1,
sc.pl.umap(tcell_ann_IE, color=['leiden_48',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
#sc.pp.neighbors(tcell_ann_LP, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)
#sc.tl.leiden(tcell_ann_LP,resolution=0.1)#resolution=0.1,
sc.pl.umap(tcell_ann_LP, color=['leiden_48',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
tcell_ann_LP_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_LP.h5ad'
tcell_ann_LP.write(tcell_ann_LP_loc)

tcell_ann_IE_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_IE.h5ad'
tcell_ann_IE.write(tcell_ann_IE_loc)

In [ ]:
DoublePos.obs.index

In [ ]:
tcell_ann_concat.obs['leiden_48'] = tcell_ann_concat.obs['leiden_48'].cat.add_categories('Double_Pos')

In [ ]:
tcell_ann_concat.obs.loc[tcell_ann_concat.obs.index.isin(DoublePos.obs.index),'leiden_48']= "Double_Pos"

In [ ]:
sc.pl.umap(tcell_ann_concat, color='leiden_48', legend_loc='on data', title='', frameon=False, save='.pdf', size=15)

In [ ]:
tcell_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_concat_Tcell_only.h5ad'
tcell_ann_concat.write(tcell_ann_loc)

In [ ]:
tcell_ann_concat.obs['CD4_8'] = tcell_ann_concat.obs['leiden']

# Annotation

In [ ]:
tcell_ann_concat = anndata.read_h5ad('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/tcell_ann_concat_Tcell_only.h5ad')

In [ ]:
tcell_ann_concat

In [ ]:
#tcell_ann_concat.write_loom('/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_concat_Tcell_only.loom', write_obsm_varm=True)
#!pip install loompy

In [ ]:
#sc.tl.leiden(tcell_ann_concat,resolution=1.5)#resolution=0.1,
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'],legend_loc='on data',ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=["CD3D","CD3E","CD3G"], ncols=2)

In [ ]:
tcell_ann_concat

In [ ]:
tcell_ann_concat.obsm

In [ ]:
tcell_ann_concat

In [ ]:
##### Update some meta data infromation
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID']=='C179','GVHD_Grade_1'] = "Mild"
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID']=='R085','GVHD_Grade_1'] = "No"

tcell_ann_concat.obs['GVHD_Grade_1'] = tcell_ann_concat.obs['GVHD_Grade_1'].astype('category')
#tcell_ann_concat.obs['GVHD_Grade_1'] = tcell_ann_concat.obs['GVHD_Grade_1'].cat.add_categories(['Treated']) ### Let us call it No instead
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['Days_post-Transplant']==444.0,'GVHD_Grade_1'] = "No"

tcell_ann_concat.obs['PatientID_Time'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['Days_post-Transplant'].astype(str)

sc.pl.umap(tcell_ann_concat, color=['PatientID_Time'],ncols=2)

In [ ]:
tcell_ann_concat.obs['PatientID_Tissue_Loc'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)

tcell_ann_concat.obs['PatientID_Tissue_Loc'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)

# Update Location specific or Day specific meta data
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID_Tissue_Loc']=='C162_Upper GI','GVHD_Grade_1'] = "No"
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID_Time']=='C160_28.0','GVHD_Grade_1'] = "Mild"
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID_Time']=='C160_196.0','GVHD_Grade_1'] = "No"

tcell_ann_concat.obs['PatientID_Grade'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)

sc.pl.umap(tcell_ann_concat, color=['PatientID_Tissue_Loc'],ncols=2)

In [ ]:
tcell_ann_concat.obs['PatientID_Grade_Time_Loc'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat.obs['Days_post-Transplant'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['PatientID_Grade_Time_Loc'],ncols=2)

In [ ]:
plt.rcParams.update({'font.size':10})
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident \nMemory T cells', "CD4+ \nCentral Memory T cells","CD8+ Cytotoxic \nUnconventional T cells",'Low CD3',"CD8+ \nProliferating T cells","CD4+ Regulatory T cells","CD8+ Homeostatic \nUnconventional T cells ", "CXCR3 High \nResident CD8 T cells"]
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

sc.pl.umap(tcell_ann_concat, color='leiden_0.5', legend_loc='on data', title='', frameon=False, save='.pdf')

In [ ]:
### Remove small clusters
# Get the value counts
cluster_counts = tcell_ann_concat.obs['leiden_0.5'].value_counts()

# Filter out clusters with counts lower than 200
filtered_clusters = cluster_counts[cluster_counts >= 200]

# Get the names of the clusters to keep
clusters_to_keep = filtered_clusters.index

# Filter the AnnData object to keep only the cells in those clusters
filtered_tcell_ann_concat = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'].isin(clusters_to_keep)]

# Optionally, view the filtered counts
filtered_counts = filtered_tcell_ann_concat.obs['leiden_0.5'].value_counts()
filtered_counts

tcell_ann_concat = filtered_tcell_ann_concat

# Figure 4C

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (4, 3)  # Set default figure size to 6×4 inches
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size


# New cluster names
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident \nMemory T cells', "CD4+ \nCentral Memory T cells","CD8+ Cytotoxic \nUnconventional T cells",'Low CD3',"CD8+ \nProliferatingT cells","CD4+ Regulatory T cells",'CD8+ Homeostatic \nUnconventional T cells '
]

# Rename clusters
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

# Generate UMAP plot with larger dots
sc.pl.umap(
    tcell_ann_concat,
    color='leiden_0.5',
    legend_loc='on data',
    title='',
    frameon=False,
    size=10,  # Adjust this value to make the dots bigger (default is 120)
    show=False  # We will save it manually
)

# Save the plot manually with DPI 300
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300.pdf', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig('//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300.png', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig('//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300.svg', dpi=300, bbox_inches='tight', transparent = True)
# Show the plot (optional, if you want to see it before saving)
plt.show()

In [ ]:
###### Filter out samples with low cell count
# Calculate cell counts per sample
cell_counts = tcell_ann_concat.obs['SampleID'].value_counts()

# Get samples with at least 100 cells
valid_samples = cell_counts[cell_counts >= 100].index

# Subset AnnData to retain only valid samples
tcell_ann_concat_f = tcell_ann_concat[tcell_ann_concat.obs['SampleID'].isin(valid_samples), :]

In [ ]:
# Update global font size
for cell_type in tcell_ann_concat_f.obs['LPorIE'].unique():

    
    # Set the figure size manually
    plt.figure(figsize=(12, 10))  # Adjust the figure size if needed

    # Generate UMAP plot with larger dots
    sc.pl.umap(
        tcell_ann_concat_f[tcell_ann_concat_f.obs['LPorIE']==cell_type],
        color='leiden_0.5',
        legend_loc='on data',
        title='',
        frameon=False,
        size=10,  # Adjust this value to make the dots bigger (default is 120)
        show=False  # We will save it manually
    )
    
    # Save the plot manually with DPI 300
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_{cell_type}_dpi300.pdf', dpi=300, bbox_inches='tight', transparent = True)
    plt.savefig(f'//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_{cell_type}_dpi300.png', dpi=300, bbox_inches='tight', transparent = True)
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_{cell_type}_dpi300.svg', dpi=300, bbox_inches='tight', transparent = True)
    # Show the plot (optional, if you want to see it before saving)
    plt.show()

In [ ]:
# Generate UMAP plot with larger dots
sc.pl.umap(
    tcell_ann_concat_f[tcell_ann_concat_f.obs['leiden_0.5'].isin(['CD4+ Effector T cells',"CD4+ Regulatory T cells",  "CD4+ \nCentral Memory T cells"])],
    color='leiden_0.5',
    legend_loc='on data',
    title='',
    frameon=False,
    size=10,  # Adjust this value to make the dots bigger (default is 120)
    show=False  # We will save it manually
)

# Save the plot manually with DPI 300
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_dpi300.pdf', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig(f'//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_dpi300.png', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_dpi300.svg', dpi=300, bbox_inches='tight', transparent = True)
# Show the plot (optional, if you want to see it before saving)
plt.show()

In [ ]:

# Define the order you want
desired_order = ['ND','No', 'Mild', 'Severe']  # Adjust based on your data

# Convert to categorical with specified order
tcell_ann_concat_f.obs["GVHD_Grade_1"] = pd.Categorical(
    tcell_ann_concat_f.obs["GVHD_Grade_1"],
    categories=desired_order,
    ordered=True
)
tcell_ann_concat_f.uns["GVHD_Grade_1_colors"] = ['#09BB8C','#09BB8C','#005A8F', '#B85000']
# Generate UMAP plot with larger dots
sc.pl.umap(
    tcell_ann_concat_f[tcell_ann_concat_f.obs['leiden_0.5'].isin(['CD4+ Effector T cells',"CD4+ Regulatory T cells",  "CD4+ \nCentral Memory T cells"])],
    color='GVHD_Grade_1',
    legend_loc='right',
    title='',
    frameon=False,
    size=10,  # Adjust this value to make the dots bigger (default is 120)
    show=False  # We will save it manually
)

# Save the plot manually with DPI 300
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_grade_dpi300.pdf', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig(f'//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_grade_dpi300.png', dpi=300, bbox_inches='tight', transparent = True)
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_cd4_grade_dpi300.svg', dpi=300, bbox_inches='tight', transparent = True)
# Show the plot (optional, if you want to see it before saving)
plt.show()

In [ ]:
# Update global font size
plt.rcParams.update({'font.size': 14})  # This sets a better default font size

# Define CD4+ T cell subsets of interest
cd4_subsets = ['CD4+ Effector T cells', 'CD4+ Regulatory T cells', 'CD4+ \nCentral Memory T cells']

# Loop through each location (LP or IE)
for cell_type in tcell_ann_concat_f.obs['LPorIE'].unique():
    
    # Create a boolean mask for filtering
    location_mask = tcell_ann_concat_f.obs['LPorIE'] == cell_type
    subset_mask = tcell_ann_concat_f.obs['leiden_0.5'].isin(cd4_subsets)
    combined_mask = location_mask & subset_mask  # Proper Boolean operation
    
    # Filter the AnnData object
    filtered_data = tcell_ann_concat_f[combined_mask]
    
    # Set the figure size manually
    plt.figure(figsize=(12, 10))
    
    # Generate UMAP plot with larger dots
    sc.pl.umap(
        filtered_data,
        color='leiden_0.5',
        legend_loc='on data',
        title=f'CD4+ T Cell Subsets in {cell_type}',
        frameon=False,
        size=10,  # This is the scanpy default size parameter (not matplotlib's)
        show=False  # We will save it manually
    )
    
    # Create output directory if it doesn't exist
    output_dir = '/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/'
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the plot in multiple formats
    base_filename = f'{output_dir}cd4_umap_clusters_{cell_type}_dpi300'
    plt.savefig(f'{base_filename}.pdf', dpi=300, bbox_inches='tight', transparent=True)
    plt.savefig(f'{base_filename}.png', dpi=300, bbox_inches='tight', transparent=True)
    plt.savefig(f'{base_filename}.svg', bbox_inches='tight', transparent=True)  # SVG doesn't need DPI
    
    # Show the plot
    plt.show()
    
    # Close the figure to free memory
    plt.close()

# Print summary of processed data
print(f"Generated UMAP plots for {len(tcell_ann_concat_f.obs['LPorIE'].unique())} tissue locations")
print(f"Included CD4+ T cell subsets: {', '.join(cd4_subsets)}")

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

# Update global font size
plt.rcParams.update({'font.size': 10})

# New cluster names
new_cluster_names = [
    'CD8+ Effector T cells', 'CD4+ Effector T cells', 
    '', "CD4+ \nCentral Memory T cells",
    "CD8+ Cytotoxic \nUnconventional T cells", 'Low CD3', 
    "CD8+ \nProliferating T cells", "CD4+ Regulatory T cells",
    ' '
]

# Rename clusters
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

# Set the figure size manually
plt.figure(figsize=(12, 10))  # Adjust the figure size if needed

# Generate UMAP plot with larger dots and transparency
sc.pl.umap(
    tcell_ann_concat,
    color='leiden_0.5',
    legend_loc='on data',
    title='',
    frameon=False,
    size=15,  # Adjust this value to make the dots bigger (default is 120)
    alpha=0.6,  # Add transparency to the dots (range is 0 to 1)
    show=False  # We will save it manually
)

# Save the plot manually with DPI 300
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation.pdf', dpi=300, bbox_inches='tight')
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation.png', dpi=300, bbox_inches='tight')
# Show the plot (optional, if you want to see it before saving)
plt.show()


In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

# Update global font size
plt.rcParams.update({'font.size': 10})

# New cluster names
# New cluster names
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident \nMemory T cells', "CD4+ \nCentral Memory T cells","CD8+ Cytotoxic \nUnconventional T cells",'Low CD3',"Proliferating \nCD8+ T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic \nUnconventional T cells '
]

# Rename clusters
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

# Set the figure size manually
plt.figure(figsize=(12, 10))  # Adjust the figure size if needed

# Generate UMAP plot with larger dots and transparency
sc.pl.umap(
    tcell_ann_concat,
    color='leiden_0.5',
    title='',
    frameon=False,
    size=15,  # Adjust this value to make the dots bigger (default is 120)
    alpha=0.6,  # Add transparency to the dots (range is 0 to 1)
    show=False  # We will save it manually
)

plt.gca().legend(
    prop={'weight': 'bold'},  # Make the legend text bold
    loc='best',  # Adjust legend location if necessary
    frameon=True  # Add a frame around the legend (optional)
)

# Save the plot manually with DPI 300
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation_legend.pdf', dpi=300, bbox_inches='tight')
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation_legend.png', dpi=300, bbox_inches='tight')
# Show the plot (optional, if you want to see it before saving)
plt.show()


In [ ]:
tcell_ann_concat.uns['leiden_0.5_colors']

In [ ]:
tcell_ann_concat.obs['leiden_0.5']

# Figure 4F

In [ ]:
category_order = ['ND', 'No', 'Mild', 'Severe']

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 2.5)  # Set default figure size to 6×4 inches
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

# New cluster names
new_cluster_names = [
    'CD8+ Effector T cells', 'CD4+ Effector T cells', 
    'CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells",
    "CD8+ Cytotoxic Unconventional T cells", 'Low CD3', 
    "CD8+ Proliferating T cells", "CD4+ Regulatory T cells",
    'CD8+ Homeostatic Unconventional T cells'
]

tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

for cat in category_order:
    print(cat)
    # Generate UMAP plot with larger dots and transparency
    sc.pl.umap(
        tcell_ann_concat[tcell_ann_concat.obs['GVHD_Grade_1']== cat],
        color='leiden_0.5',
        title='',
        frameon=False,
        size=8,  # Adjust this value to make the dots bigger (default is 120)
        show=False,  # We will save it manually
        legend_loc=None
    )

    # Save the plot manually with DPI 300
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300_presentation_{cat}.pdf', dpi=300, bbox_inches='tight', transparent = True)
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300_presentation_{cat}.png', dpi=300, bbox_inches='tight', transparent = True)
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/umap_clusters_dpi300_presentation_{cat}.svg', dpi=300, bbox_inches='tight', transparent = True)
# Show the plot (optional, if you want to see it before saving)
    plt.show()

In [ ]:
sc.pl.umap(
        tcell_ann_concat[tcell_ann_concat.obs['GVHD_Grade_1'].isin(['No', 'Mild', 'Severe'])],
        color='leiden_0.5',
        title='',
        frameon=False,
        size=15,  # Adjust this value to make the dots bigger (default is 120)
        show=False,  # We will save it manually
    )

    # Save the plot manually with DPI 300
plt.savefig(f'/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation_GVHD.pdf', dpi=300, bbox_inches='tight')
plt.savefig(f'/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/umap_clusters_dpi300_presentation_GVHD.png', dpi=300, bbox_inches='tight')
# Show the plot (optional, if you want to see it before saving)
plt.show()

In [ ]:
for cat in category_order:
    print(cat)
    sc.pl.umap(tcell_ann_concat[tcell_ann_concat.obs['GVHD_Grade_1']== cat], color=['leiden_0.5'],legend_loc='on data',ncols=2)

In [ ]:
# Dr. Reshef
Ran_list = ['ID2',
'ID3',
'PRDM1',
'EOMES',
'IL21R',
'IL12RB1',
'IL12RB2',
'NFE2L2',
'BATF2',
'TBX21',
'NR3C1']

sc.pl.umap(tcell_ann_concat, color=['ID2',
'ID3',
'PRDM1',
'EOMES',
'IL21R',
'IL12RB1',
'IL12RB2',
'NFE2L2',
'BATF2',
'TBX21',
'NR3C1'], ncols=2)



In [ ]:
sc.pl.umap(tcell_ann_concat, color=["CD3D","CD3E","CD3G", 'CD4', 'CD8A','FOXP3','TOX','CD38','LAG3','HAVCR2','GZMK','GZMH'], ncols=2)

#EFFECTOR
sc.pl.umap(tcell_ann_concat, color=['PRF1','TBX21','ZNF683','ZEB2','GZMB','GZMM'], ncols=2)

# Activated CD8

In [ ]:
#cd8 cytotoxic
sc.pl.umap(tcell_ann_concat, color=['FASLG','CTSW','CD44','CCR7'], ncols=2)

In [ ]:
#CD8 EXHAUSTED
sc.pl.umap(tcell_ann_concat, color=['TOX','CD38','LAG3','HAVCR2', 'TIGIT','PDCD1','EOMES', 'CTLA4'], ncols=2)

In [ ]:
# CD8 activated
sc.pl.umap(tcell_ann_concat, color=['CCL4','CCL3','CCL5','CXCR3','CST7','DUSP2','KLRG1','NKG7'], ncols=2)


In [ ]:
sc.pl.umap(tcell_ann_concat, color=['IL2RB'], ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['CDH17','KIR2DL4','KLHL23','S1PR3','FCRL4'], ncols=2)

# Non classical antigen presenting T cells

In [ ]:
#th1
sc.pl.umap(tcell_ann_concat, color=['TBX21','IFNG','STAT4','STAT1','TNF','CXCR3','IL2','IL12RB1','IL12RB2','IL18R1','LTA','LTB'], ncols=2)

In [ ]:
#th17
sc.pl.umap(tcell_ann_concat, color=['RORC','IL17A','IL17F','AHR','IKZF3','STAT3','IL23R','IKZF1','CCR6','IL21','IL7R'], ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['TRAC','TRDC','TRBC1','ZAP70','LCK'], ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['CXCR3','CXCR4','CXCR6','CCR9','CX3CR1','CXCR1','CXCR2','CCR10','S1PR3','S1PR4','S1PR5'], ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['IL32'], ncols=2)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['CD8A','CD8B'], ncols=2)

In [ ]:
#Central memnory marker
sc.pl.umap(tcell_ann_concat, color=['PTPRC'], ncols=2)

In [ ]:
#Tissue resident
sc.pl.umap(tcell_ann_concat, color=['ITGAE'], ncols=2)

embedding=tcell_ann_concat.obsm["X_umap"]
embedding

A = kneighbors_graph(embedding,150, mode="connectivity", include_self=True)
G = nx.from_numpy_matrix(A)
partition = community_louvain.best_partition(G)

tcell_ann_concat.obs["Louvain_str"] = [str(x) for x in list(partition.values())]
tcell_ann_concat.obs["Louvain"] = list(partition.values())
tcell_ann_concat

sc.pl.umap(tcell_ann_concat, color=["Louvain_str"])

sc.pl.umap(tcell_ann_concat, color=["Louvain_str"])

In [ ]:
#sc.pl.umap(tcell_ann_concat, color=["LAG3",'CTLA4','TOX','PDCD1','TIGIT','HAVCR2','CD244'])
sc.pl.umap(tcell_ann_concat, color=['CD44','ITGAE','CD69','LPorIE',"leiden_48"])

In [ ]:
#tcell_ann_concat.obs['CD4_8']= tcell_ann_concat.obs['CD48']

In [ ]:
#sc.pl.umap(tcell_ann_concat, color='CD4_8', legend_loc='on data')

In [ ]:
#saved no need to run again
sc.tl.rank_genes_groups(tcell_ann_concat, groupby="leiden_0.5", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(tcell_ann_concat, group=None)
dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_Tcells_leiden_0.5_w_notraw.csv')

In [ ]:
sorter = ['C155','C101','C162','C157','R085','C160','C159','C152','C068','C98','C179','C66','ND001','ND003']

In [ ]:
from copy import deepcopy
#savepath = '/Users/lingtingshi/Documents/Azizi_lab/GVHD/'
obs_1 = 'PatientID'
obs_2 = 'leiden_48'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
#df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True)
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True)
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


In [ ]:
savepath = '/burg/iicd/users/ls3456/GVHD/'
obs_1 = 'GVHD_Grade_1'
obs_2 = 'leiden_48'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
plt.savefig(savepath + '/' + obs_1 + '_stacked_barplot_color.pdf',
            transparent=True, bbox_inches='tight', pad_inches=0.02)

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color =tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
plt.savefig(savepath + '/' + obs_2 + '_stacked_barplot.pdf',
            transparent=True, bbox_inches='tight', pad_inches=0.02)

In [ ]:
sc.pl.umap(tcell_ann_concat, color='PatientID')

In [ ]:
tcell_ann_concat.uns['PatientID_colors']

In [ ]:
sc.pl.umap(tcell_ann_concat, color='leiden_0.5')

# Plot T cell signatures 

In [ ]:
markers_df = pd.read_csv('/burg/iicd/users/ls3456/GVHD/GVHD_markers_T cells.csv')
markers_df.columns

In [ ]:
select_sigs = ['Naïve T cells','Terminally Differentiated','Tregs','Pro \ninflammatory','Cytolytics \neffector pathway','Exhaustion/Terminal \ndifferentiation', 'CD8+ peripheral \ntolerance','Th17.1','CD8 T Cell Activation','TCell Exhaustion','Gamma_T_cell_Noa', 'CD8 T cell \nactivation','Terminal\nexhaustion','Precursor\nexhaustion']

In [ ]:
#cell_dic
select_sigs = markers_df.columns

markers_df = markers_df[select_sigs]

cells = markers_df.columns.values

marker_genes = []
cell_dic_plot = {}
for c in cells:
    cc = np.array(markers_df[c].values.tolist())
    marker_genes = np.append(marker_genes,[x for x in cc if str(x) != 'nan'])
    cell_dic_plot[c] = [x for x in cc if str(x) != 'nan']

In [ ]:
marker_genes

In [ ]:
embedding=tcell_ann_concat.obsm["X_umap"]
embedding

In [ ]:
plt.figure(figsize=(6, 5)) 

for i in cell_dic_plot.keys():
    print(i)
    genes = cell_dic_plot[i]
    genes_up = []
    for g in genes:
        genes_up = np.append(genes_up,g.upper())

    select_genes_pos = []
    select_genes_neg = []

    for gene in genes_up:
        if "-" in gene and "+" in gene:
            continue
        elif "-" in gene:
            select_genes_neg.append(gene.split("-")[0].strip())
        else:
            select_genes_pos.append(gene.split("+")[0].strip())

    genes_up_pos = np.array(select_genes_pos)
    genes_up_neg = np.array(select_genes_neg)

    # get intersection
    top_genes_pos = np.intersect1d(tcell_ann_concat.var.index, genes_up_pos)
    top_genes_neg = np.intersect1d(tcell_ann_concat.var.index, genes_up_neg)
    sc.pl.umap(tcell_ann_concat, color=top_genes_pos)
    if len(top_genes_neg) >0:
        sc.pl.umap(tcell_ann_concat, color=top_genes_neg)

    val_pos = np.sum(tcell_ann_concat.to_df().loc[:,top_genes_pos],axis=1) # sum for each row across columns
    val_pos = np.array(val_pos).flatten()

    val_neg = np.sum(tcell_ann_concat.to_df().loc[:,top_genes_neg] * -1,axis=1) # sum for each row across columns
    val_neg = np.array(val_neg).flatten()

    val = val_neg + val_pos

    label = i
    tcell_ann_concat.obs[label] = val/(len(top_genes_pos)+len(top_genes_neg))

    f, ax = plt.subplots()

    #sc.pl.scatter(tcell_ann,color=label,legend_loc='none',basis='umap',color_map='Spectral_r')
    points = ax.scatter(embedding[:, 0], embedding[:, 1], c=tcell_ann_concat.obs[label].values, s=2, edgecolors='none')
    plt.title(i)
    f.colorbar(points)

    plt.show()

In [ ]:
plt.figure(figsize=(3, 2)) 
# Naive Resting
sc.pl.umap(tcell_ann_concat, color=['CXCR3'], use_raw=False)

In [ ]:
plt.figure(figsize=(3, 2)) 
# Naive Resting
sc.pl.umap(tcell_ann_concat, color=['CCR7', 'SELL'])

In [ ]:
# Treg
sc.pl.umap(tcell_ann_concat, color=['CTLA4', 'FOXP3', 'ICOS','TIGIT','IL2RA','IKZF2'])

In [ ]:
# TH17
sc.pl.umap(tcell_ann_concat, color=['IL2', 'IL23R', 'RORC','IL7R','KLRB1','CD40LG','NFKBIA'], ncols = 2)

In [ ]:
# TH2
sc.pl.umap(tcell_ann_concat, color=['GATA3', 'IL13','IL5' ])

In [ ]:
# T cell activation marker
sc.pl.umap(tcell_ann_concat, color=['IL2RA','CD69','TNFRSF4','CD40LG','LTB','ICOS'], ncols=2)



In [ ]:
# TCR makers
sc.pl.umap(tcell_ann_concat, color=['TRGV4','TRGV5','TRGV8','TRGV3','TRGV2','TRGV7','TRGV9','TRGC2'], ncols=2)

###### 

In [ ]:
# Memory makers
sc.pl.umap(tcell_ann_concat, color=['CD7','XCL1','CAPG','CD160'], ncols=3)



In [ ]:
# Proliferating CD8+ T cells
sc.pl.umap(tcell_ann_concat, color=['MKI67','TYMS','MCM7'], ncols=3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['TCF7','RUNX3','PRDM1','AHR','TBX21','EOMES','ID3','KLF2','ZNF683'], ncols=3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['CCR7','EOMES','ZNF683','S1PR1'], ncols=3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['GZMK','GZMH'], ncols=3)

In [ ]:
x = tcell_ann_concat[:, 'GZMK'].X
y = tcell_ann_concat[:, 'GZMH'].X

# If your AnnData is sparse, convert to dense first
if not isinstance(x, np.ndarray):
    x = x.toarray().flatten()
if not isinstance(y, np.ndarray):
    y = y.toarray().flatten()

# Scatter plot
plt.figure(figsize=(6,6))
plt.scatter(x, y, alpha=0.5, s=10)  # s is point size
plt.xlabel('GZMK expression')
plt.ylabel('GZMH expression')
plt.title('GZMK vs GZMH expression')
plt.grid(True)
plt.show()

In [ ]:
# Extract expression for GZMK and GZMH
x = tcell_ann_concat[:, 'GZMK'].X
y = tcell_ann_concat[:, 'GZMH'].X

# If your AnnData is sparse, convert to dense first
if not isinstance(x, np.ndarray):
    x = x.toarray().flatten()
if not isinstance(y, np.ndarray):
    y = y.toarray().flatten()

import seaborn as sns
import matplotlib.pyplot as plt

# Filter: keep cells where GZMK > 0 or GZMH > 0
mask = (x > 0) | (y > 0)
x_filtered = x[mask]
y_filtered = y[mask]

# Then plot density
plt.figure(figsize=(6,6))
sns.kdeplot(
    x=x_filtered, y=y_filtered,
    fill=True,
    thresh=0.05,
    levels=100,
    cmap="mako"
)
plt.xlabel('GZMK expression')
plt.ylabel('GZMH expression')
plt.title('Density of GZMK vs GZMH expression (at least one positive)')
plt.grid(True)
plt.show()

In [ ]:
select_sigs = list(select_sigs)
select_sigs.append('leiden_0.5')

In [ ]:
list(select_sigs)

In [ ]:
tcell_ann_concat

In [ ]:
table = pd.pivot_table(tcell_ann_concat.obs[select_sigs],index ='leiden_0.5',aggfunc='mean')

In [ ]:
table

In [ ]:
plt.figure(figsize=(16, 6))
sns.heatmap(table)

In [ ]:
z_scores = zscore(table, axis= 0)

In [ ]:
z_scores

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5', 'CD3D'])

In [ ]:
plt.figure(figsize=(34, 6))
sns.heatmap(z_scores)

In [ ]:
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(15, 6))

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'])

# CD4 Reclustering

In [ ]:
cd4effector = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5']== "CD4+ Effector T cells"]
sc.tl.rank_genes_groups(cd4effector, groupby="GVHD_Grade_1", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(cd4effector, group=None)
dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_cd4effector.csv')
dedf = sc.get.rank_genes_groups_df(cd4effector, group="Mild")
dedf.head(30)
dedf = sc.get.rank_genes_groups_df(cd4effector, group="Severe")
dedf.head(30)

In [ ]:
CD4cm = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5']== 'CD4+\nCentral Memory']
CD4cm

In [ ]:
CD4ef = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5']== 'Effector\nCD4+ T-cells']
CD4ef

In [ ]:
#sc.pp.neighbors(CD4cm, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)
sc.pl.umap(CD4cm, color = 'leiden_0.5')

In [ ]:
sc.tl.leiden(CD4cm,resolution=0.3, key_added= "leiden")#,resolution=0.1,
sc.pl.umap(CD4cm,color = 'leiden',legend_loc = 'on data',legend_fontsize = 'xx-small')

In [ ]:
#th1
sc.pl.umap(CD4cm, color=['TBX21','IFNG','STAT4','STAT1','TNF','CXCR3','IL2','IL12RB1','IL12RB2','IL18R1','LTA','LTB'], ncols=3)

#th17
sc.pl.umap(CD4cm, color=['RORC','IL17A','IL17F','AHR','IKZF3','STAT3','IL23R','IKZF1','CCR6','IL21','IL7R'], ncols=4)

# TH2
sc.pl.umap(CD4cm, color=['GATA3', 'IL13','IL5' ])

# T cell activation marker
sc.pl.umap(CD4cm, color=['IL2RA','CD69','TNFRSF4','CD40LG','LTB','ICOS'], ncols=2)


# Naive Resting
sc.pl.umap(CD4cm, color=['CCR7', 'SELL'])

In [ ]:
sc.tl.rank_genes_groups(CD4cm, groupby="leiden", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(CD4cm, group=None)
dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_CD4cm_leiden_w_notraw.csv')

In [ ]:
sc.pl.umap(CD4ef,color = 'leiden_0.5',legend_loc = 'on data',legend_fontsize = 'xx-small')

In [ ]:
#sc.pp.neighbors(CD4ef, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)
#sc.tl.umap(CD4ef)
sc.tl.leiden(CD4ef,resolution=0.3, key_added= "leiden")#,resolution=0.1,
sc.pl.umap(CD4ef,color = 'leiden',legend_loc = 'on data',legend_fontsize = 'xx-small')

In [ ]:
#th1
sc.pl.umap(CD4ef, color=['TBX21','IFNG','STAT4','STAT1','TNF','CXCR3','IL2','IL12RB1','IL12RB2','IL18R1','LTA','LTB'], ncols=3)

#th17
sc.pl.umap(CD4ef, color=['RORC','IL17A','IL17F','AHR','IKZF3','STAT3','IL23R','IKZF1','CCR6','IL21','IL7R'], ncols=4)

# TH2
sc.pl.umap(CD4ef, color=['GATA3', 'IL13','IL5' ])

# T cell activation marker
sc.pl.umap(CD4ef, color=['IL2RA','CD69','TNFRSF4','CD40LG','LTB','ICOS'], ncols=2)

# Naive Resting
sc.pl.umap(CD4ef, color=['CCR7', 'SELL'])

In [ ]:
sc.tl.rank_genes_groups(CD4ef, groupby="leiden", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(CD4ef, group=None)
dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_CD4ef_leiden_w_notraw.csv')

# Heatmaps and dot plots for each cluster

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"CD8+ Proliferating T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells'
]
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

In [ ]:
# Replace 'CATEGORY_NAME' with the actual name of the categorical field in .obs
category_name = 'leiden_0.5'

# Check if the color map exists in `.uns`
if f"{category_name}_colors" in tcell_ann_concat.uns:
    category_colors = tcell_ann_concat.uns[f"{category_name}_colors"]
    print(category_colors)
else:
    print(f"No colors found for category {category_name}.")

# Extract the category values
categories = tcell_ann_concat.obs[category_name].cat.categories

# Pair colors with category values
color_map = dict(zip(categories, category_colors))
print(color_map)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'])

In [ ]:
# Read the CSV file into a DataFrame
dedf = pd.read_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_Tcells_leiden_0.5_w_notraw.csv')

# Display the first few rows of the DataFrame to confirm it loaded correctly
dedf

In [ ]:
# Map the cluster labels (0-9) to the new cluster names
dedf['cluster_name'] = dedf['group'].map({i: name for i, name in enumerate(new_cluster_names)})

# Display the first few rows of the DataFrame to confirm the changes
dedf

In [ ]:
import pandas as pd

# Assuming dedf is your DataFrame and it contains columns 'pval_adj', 'log_fold_change', and 'cluster_name'

# Filter the DataFrame by pval_adj < 0.05
filtered_dedf = dedf[dedf['pvals_adj'] < 0.05]

# Sort the DataFrame by 'cluster_name' and 'log_fold_change' in descending order
filtered_dedf = filtered_dedf.sort_values(by=['cluster_name', 'logfoldchanges'], ascending=[True, False])

# Select the top 300 log fold change entries for each cluster_name
top_300_per_group = filtered_dedf.groupby('cluster_name').head(500)

# Display the resulting DataFrame
top_300_per_group

In [ ]:
print(list(top_300_per_group[top_300_per_group['cluster_name'] == 'CD4+ Central Memory T cells']['names']))

In [ ]:
clusters = tcell_ann_concat.obs['leiden_0.5'].astype(str)

# Compute the mean gene expression for each gene within each cluster
cluster_means = pd.DataFrame(index=tcell_ann_concat.var_names)

for cluster in clusters.unique():
    cluster_mask = (clusters == cluster)
    cluster_data = tcell_ann_concat[cluster_mask, :]
    cluster_mean_expression = cluster_data.to_df().mean(axis=0)
    cluster_means[cluster] = cluster_mean_expression

In [ ]:
cluster_means

In [ ]:
tcell_ann_concat.obs['leiden_0.5']

In [ ]:
# Assuming 'cluster_means' and 'genes' are already defined and processed
heat_pre = cluster_means[cluster_means.index.isin(top_300_per_group['names'])]
z_scores = zscore(heat_pre, axis=1)
sns.set_style("whitegrid", {'axes.grid' : False})
# Generate the clustermap
sns.clustermap(
    z_scores, 
    cmap='viridis', 
    method='average', 
    metric='euclidean', 
    figsize=(6, 15), 
    linewidths=0  # Set linewidths to 0 to remove grid lines
)

In [ ]:
sc.tl.dendrogram(tcell_ann_concat, groupby='leiden_0.5')

In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

# Define all the marker sets
marker_sets = {
    'Cytotoxic Markers': ['PRF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'FASLG', 'SYTL2', 'CTSW'],
    'Proliferating Markers': ['MKI67', 'TOP2A', 'GTSE1', 'ASPM', 'KIF4A', 'KIF11', 'KIF14', 'KIF23', 'KNL1', 'CENPF'],
    'Treg Markers': ['FOXP3', 'IL2RA', 'IL10', 'MAF', 'AREG', 'ICOS', 'IKZF2'],
    'T Cell Markers': ['CD3E', 'CD3D', 'TRAC', 'TRBC1', 'TRBC2', 'TRDC', 'ZAP70', 'LCK', 'CD8A', 'CD8B', 'CD4'],
    'Resident Markers': ["ITGA1", "ITGAE", "ZNF683",'HOPX','CD69','CD244'],
    'Infiltrating Markers': ["ITGB2", "ITGAL", 'VCAM1'],
    'Exhaustion Markers': ['LAG3', 'PDCD1', 'CTLA4', 'TIGIT', 'HAVCR2', 'TOX', 'CD101'],
    'Unconventional Markers': ['TRDC', 'KLRC1', 'KLRC2', 'KLRC3', 'KIR2DL4', 'KIR3DL2', 'IL2RB', 'IKZF2', 'CDH17'],
    'Activation Markers': ['CCL5', 'CCR5', 'CXCR3', 'CST7', 'CD69', 'DUSP2', 'KLRG1', 'AOAH'],
    'CD8 Activation': ['CD69', 'CCR7', 'CD27', 'BTLA', 'CD40LG', 'IL2RA', 'CD3E', 'CD47', 'EOMES', 'GNLY', 'GZMA', 'GZMB', 'PRF1', 'IFNG', 'CD8A', 'CD8B', 'LAMP1', 'LAG3', 'CTLA4', 'HLA-DRA', 'TNFRSF4', 'ICOS', 'TNFRSF9', 'TNFRSF18'],
    'Central Memory Markers': ['SELL', 'CCR7','PTPRC', 'IL2RA', 'IL7R', 'CD28', 'FAS', 'IL2RA'],
    'Central Memory 2': ['CCR7', 'SELL', 'TCF7', 'KLF2', 'CD40LG', 'CXCR5', 'MAF','CD28', "TNFRSF4"],
    'Effector Markers': ['SELL', 'CCR7', 'CD8A', 'IL2RA', 'IL7R', 'IL2', 'IFNG', 'TNF', 'HLA-DRA', 'CCR5', 'TBX21', 'GZMA'],
    'Effector Markers 2': ['GATA3', 'IL2', 'TNF', 'IL22', 'IL23R', 'IL7R', 'CCR6', 'CD40LG', 'IL2RA']
}

# Convert to DataFrame (not necessary for plotting but useful for organization)
markers_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in marker_sets.items()]))

# Plot each marker set one at a time
for marker_set, markers in marker_sets.items():
    sc.pl.dotplot(
        tcell_ann_concat, 
        var_names=markers, 
        groupby='leiden_0.5', 
        dendrogram=True, 
        title=f'{marker_set} Expression in T Cell Clusters'
    )
    plt.show()  # This ensures each plot is shown separately


In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import os

# Set figure parameters for high-resolution output
sc.settings.set_figure_params(dpi_save=300)

# Ensure the save directory exists
save_dir = '/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures'
os.makedirs(save_dir, exist_ok=True)

# Define all the marker sets
marker_sets = {
    'Cytotoxic Markers': ['PRF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'FASLG', 'SYTL2', 'CTSW'],
    'Proliferating Markers': ['MKI67', 'TOP2A', 'GTSE1', 'ASPM', 'KIF4A', 'KIF11', 'KIF14', 'KIF23', 'KNL1', 'CENPF'],
    'Treg Markers': ['FOXP3', 'IL2RA', 'IL10', 'MAF', 'AREG', 'ICOS', 'IKZF2'],
    'T Cell Markers': ['CD3E', 'CD3D', 'TRAC', 'TRBC1', 'TRBC2', 'TRDC', 'ZAP70', 'LCK', 'CD8A', 'CD8B', 'CD4'],
    'Resident Markers': ["ITGA1", "ITGAE", "ZNF683",'HOPX','CD69','CD244'],
    'Infiltrating Markers': ["ITGB2", "ITGAL", 'VCAM1'],
    'Exhaustion Markers': ['LAG3', 'PDCD1', 'CTLA4', 'TIGIT', 'HAVCR2', 'TOX', 'CD101'],
    'Unconventional Markers': ['TRDC', 'KLRC1', 'KLRC2', 'KLRC3', 'KIR2DL4', 'KIR3DL2', 'IL2RB', 'IKZF2', 'CDH17'],
    'Activation Markers': ['CCL5', 'CCR5', 'CXCR3', 'CST7', 'CD69', 'DUSP2', 'KLRG1', 'AOAH'],
    'CD8 Activation': ['CD69', 'CCR7', 'CD27', 'BTLA', 'CD40LG', 'IL2RA', 'CD3E', 'CD47', 'EOMES', 'GNLY', 'GZMA', 'GZMB', 'PRF1', 'IFNG', 'CD8A', 'CD8B', 'LAMP1', 'LAG3', 'CTLA4', 'HLA-DRA', 'TNFRSF4', 'ICOS', 'TNFRSF9', 'TNFRSF18'],
    'Central Memory Markers': ['SELL', 'CCR7','PTPRC', 'IL2RA', 'IL7R', 'CD28', 'FAS', 'IL2RA'],
    'Central Memory 2': ['CCR7', 'SELL', 'TCF7', 'KLF2', 'CD40LG', 'CXCR5', 'MAF','CD28', "TNFRSF4",'IL7R'],
    'Effector Markers': ['SELL', 'CCR7', 'CD8A', 'IL2RA', 'IL7R', 'IL2', 'IFNG', 'TNF', 'HLA-DRA', 'CCR5', 'TBX21', 'GZMA'],
    'Effector Markers 2': ['GATA3', 'IL2', 'TNF', 'IL22', 'IL23R', 'IL7R', 'CCR6', 'CD40LG', 'IL2RA']
}

# Plot each marker set one at a time and save the figures manually
for marker_set, markers in marker_sets.items():
    # Generate the dot plot and return the figure
    dotplot = sc.pl.dotplot(
        tcell_ann_concat, 
        var_names=markers, 
        groupby='leiden_0.5', 
        dendrogram=True, 
        title=f'{marker_set} Expression in T Cell Clusters',
        return_fig=True  # Return the figure object for saving
    )
    
    # Create a full path for each figure
    filename = os.path.join(save_dir, f'{marker_set.replace(" ", "_").lower()}_dotplot.pdf')
    
    # Save the figure with the specified DPI
    dotplot.savefig(filename, dpi=300, bbox_inches='tight')
    
    # Show the plot (optional)
    plt.show()

In [ ]:
cytotoxic_marker = ['PRF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'FASLG', 'SYTL2', 'CTSW']
proliferating_marker = ['MKI67', 'TOP2A', 'GTSE1', 'ASPM', 'KIF4A', 'KIF11', 'KIF14', 'KIF23', 'KNL1', 'CENPF']
treg_marker = ['FOXP3', 'IL2RA', 'IL10', 'MAF', 'AREG', 'ICOS', 'IKZF2']
Tcell_marker = ['CD3E', 'CD3D', 'TRAC', 'TRBC1', 'TRBC2', 'TRDC', 'ZAP70', 'LCK', 'CD8A', 'CD8B', 'CD4']
resident_marker =["ITGA1", "ITGAE", "ZNF683",'HOPX','CD69','CD244']
Infiltrating_marker = ["ITGB2", "ITGAL", 'VCAM1' ]
Exhaustion_marker = ['LAG3', 'PDCD1', 'CTLA4', 'TIGIT', 'HAVCR2', 'TOX', 'CD101']
unconventional_marker = ['TRDC', 'KLRC1', 'KLRC2', 'KLRC3', 'KIR2DL4', 'KIR3DL2', 'IL2RB', 'IKZF2', 'CDH17']
Activation_marker = ['CCL5', 'CCR5', 'CXCR3', 'CST7', 'CD69', 'DUSP2', 'KLRG1', 'AOAH']
CD8_activation = ['CD69', 'CCR7', 'CD27', 'BTLA', 'CD40LG', 'IL2RA', 'CD3E', 'CD47', 'EOMES', 'GNLY', 'GZMA', 'GZMB', 'PRF1', 'IFNG', 'CD8A', 'CD8B', 'LAMP1', 'LAG3', 'CTLA4', 'HLA-DRA', 'TNFRSF4', 'ICOS', 'TNFRSF9', 'TNFRSF18']
central_memory = ['SELL', 'CCR7','PTPRC', 'IL2RA', 'IL7R', 'CD28', 'FAS', 'IL2RA']
central_memory_2 = ['CCR7', 'SELL', 'TCF7', 'KLF2', 'CD40LG', 'CXCR5', 'MAF','CD28', "TNFRSF4"]
Effector_marker = ['SELL', 'CCR7', 'CD8A', 'IL2RA', 'IL7R', 'IL2', 'IFNG', 'TNF', 'HLA-DRA', 'CCR5', 'TBX21', 'GZMA']
Effector_marker_2 = ['GATA3', 'IL2', 'TNF', 'IL22', 'IL23R', 'IL7R', 'CCR6', 'CD40LG', 'IL2RA']
# Generate the dot plot
sc.pl.dotplot(
    tcell_ann_concat, 
    var_names=cytotoxic_marker, 
    groupby='leiden_0.5', 
    dendrogram=True, 
    title='Cytotoxic Marker Expression in T Cell Clusters'
)

In [ ]:
# Generate the dot plot
sc.pl.dotplot(
    tcell_ann_concat, 
    var_names=['SELL','CCR7','CD44','PTPRC','CD69','ITGAE'], 
    groupby='leiden_0.5', 
    dendrogram=True, 
    title='Cytotoxic Marker Expression in T Cell Clusters'
)

In [ ]:
# Generate the dot plot
sc.pl.dotplot(tcell_ann_concat, var_names=cytotoxic_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=treg_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=resident_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=Infiltrating_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=Activation_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=CD8_activation, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=unconventional_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=central_memory, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=central_memory_2, groupby='leiden_0.5',dendrogram=True)

In [ ]:
sc.pl.dotplot(tcell_ann_concat, var_names=Effector_marker, groupby='leiden_0.5',dendrogram=True)

In [ ]:
tcell_ann_concat.obs['leiden_0.5'].value_counts()

In [ ]:
tcell_ann_concat 

In [ ]:
sc.pl.dotplot(
    tcell_ann_concat, 
    var_names=["FOXP3",'IL2RA','CD28','ICOS','IL7R','CD40LG','GATA3','FAS','HNRNPLL','CCL5','ITGAL','CD69','ITGAE','CXCR4','CD44','CXCR6','CD63','IFNG'],  # A smaller subset of genes
    use_raw=False, 
    groupby='leiden_0.5',
    dendrogram= True,
    swap_axes=True,  # Start with no axis swap
    # Label for the group
)


In [ ]:
sc.settings.set_figure_params(dpi_save=300)

# Plot each marker set one at a time and save the figures manually
# Generate the dot plot and return the figure
dotplot = sc.pl.dotplot(
    tcell_ann_concat, 
    var_names=["CD3E",'TRBC2','CD8A','CD4', "FOXP3",'IL2RA','CD28','ICOS','IL7R','CD40LG','GATA3','CCR6',
               "GZMA",'GZMB','PRF1','IFNG',"CCL5",'CD44','CD69','LAG3','PDCD1','CTLA4','TIGIT','MKI67','ITGA1','ITGAE','CD160','ITGB2',"KLRC2",'KIR2DL4','TRDC'], 
    groupby='leiden_0.5', 
    dendrogram=True, 
    title='Signature Genes in T Cell Clusters',
    return_fig=True,  # Return the figure object for saving
    swap_axes=True, 
)


# Save the figure with the specified DPI
dotplot.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/dotplot_cluster_sign.png', dpi=300, bbox_inches='tight')
dotplot.savefig('/burg/iicd/users/ls3456/GVHD/Figures/single_cell_figures/dotplot_cluster_sign.pdf', dpi=300, bbox_inches='tight')
# Show the plot (optional)
plt.show()

# Figure 4F

In [ ]:
import pandas as pd

# Specify the desired order
desired_order = ["CD4+ Central Memory T cells",'CD4+ Effector T cells', "CD4+ Regulatory T cells", 'CD8+ Tissue Resident Memory T cells',"CD8+ Effector T cells", "CD8+ Proliferating T cells",'CD8+ Homeostatic Unconventional T cells', "CD8+ Cytotoxic Unconventional T cells", 'Low CD3']

# Reorder the categories in the `groupby` column
tcell_ann_concat.obs['leiden_0.5_reordered'] = tcell_ann_concat.obs['leiden_0.5'].astype(pd.CategoricalDtype(categories=desired_order, ordered=True))

## For publication
import matplotlib as mpl
import matplotlib as mpl
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 12,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    #'figure.figsize': (4, 3),  # Set default figure size to 6×4 inches
    'lines.linewidth': 1.2  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['xtick.labelsize'] = 12  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 12  # Colorbar tick label size


# Plot each marker set one at a time and save the figures manually
# Generate the dot plot and return the figure
dotplot = sc.pl.dotplot(
    tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'] != 'Low CD3'], 
    var_names=["CD3E",'CD4','CD8A',"GZMA",'PRF1','CD28', 'TCF7','SELL','CD69', 'CD40LG', 'TNF', 'IL2', "FOXP3",'IL2RA','ITGAE', 'ITGA1','ZNF683', 'CD160','GZMK','GZMH','MKI67', 'TOP2A','GTSE1', 'KIR2DL4', 'IKZF2', 'TRDC', 'GNLY', 'FCER1G'], 
    groupby='leiden_0.5_reordered', 
    dendrogram=False, 
    title='',
    return_fig=True,  # Return the figure object for saving
    swap_axes=True,
    figsize=(4,7),
    dot_max = 1

)

    
# Save the figure with the specified DPI
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_cluster_sign_presentation.png', dpi=300, bbox_inches='tight')
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_cluster_sign_presentation.pdf', dpi=300, bbox_inches='tight')
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_cluster_sign_presentation.svg', dpi=300, bbox_inches='tight')
# Show the plot (optional)

plt.show()

In [ ]:
## For presentations
sc.settings.set_figure_params(dpi_save=300)

# Plot each marker set one at a time and save the figures manually
# Generate the dot plot and return the figure
dotplot = sc.pl.dotplot(
    tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'] != 'Low CD3'], 
    var_names=["CD3E",'CD4','CD8A','TCF7', 'CCR7','S1PR1','CD69', 'CD40LG', 'TNF', 'IL2', "FOXP3",'IL2RA','ITGAE', 'ITGA1','ZNF683', 'CD160',"GZMA",'PRF1','GZMK','GZMH','MKI67', 'TOP2A','GTSE1', 'KIR2DL4', 'IKZF2', 'TRDC', 'GNLY', 'FCER1G'], 
    groupby='leiden_0.5_reordered', 
    dendrogram=False, 
    title='Signature Genes in T Cell Clusters',
    return_fig=True,  # Return the figure object for saving
    swap_axes=True, 
)

# Save the figure with the specified DPI
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_cluster_sign_presentation.png', dpi=300, bbox_inches='tight')
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_cluster_sign_presentation.pdf', dpi=300, bbox_inches='tight')
# Show the plot (optional)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
mpl.rcParams.update({
    'font.size': 12,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (2.5, 5),  # Set default figure size to 6×4 inches
    'lines.linewidth': 1.2  # Set default line width to 1
})
desired_order = ["CD4+ Central Memory T cells",'CD4+ Effector T cells', "CD4+ Regulatory T cells", 'CD8+ Tissue Resident Memory T cells',"CD8+ Effector T cells", "CD8+ Proliferating T cells",'CD8+ Homeostatic Unconventional T cells', "CD8+ Cytotoxic Unconventional T cells"]

# Create a mock observation dataframe
# Replace this with your actual data from `tcell_ann_concat.obs`
obs_data = pd.DataFrame({
    'leiden_0.5_reordered': desired_order
})

print(obs_data)

# Map of categories to colors
color_map = {
    'CD8+ Effector T cells': '#1f77b4', 
    'CD4+ Effector T cells': '#ff7f0e', 
    'CD8+ Tissue Resident Memory T cells': '#2ca02c', 
    'CD4+ Central Memory T cells': '#d62728', 
    'CD8+ Cytotoxic Unconventional T cells': '#9467bd', 
    'Low CD3': '#8c564b', 
    'CD8+ Proliferating T cells': '#e377c2', 
    'CD4+ Regulatory T cells': '#7f7f7f', 
    'CD8+ Homeostatic Unconventional T cells': '#bcbd22'
}

# Set y as 1 for all
x = obs_data['leiden_0.5_reordered']
y = [1] * len(x)  # y-values all set to 1
colors = [color_map[category] for category in x]  # Map colors based on categories

# Create the plot
fig, ax = plt.subplots(figsize=(2.5, 5))
bars = ax.bar(x, y, color=colors)  # Assign bar colors

# Rotate x-axis labels, set colors, and adjust layout
ax.set_xticks(range(len(x)))
ax.set_xticklabels(x, rotation=90, ha='right')  # Rotate labels

# Color the x-axis labels individually
for label, color in zip(ax.get_xticklabels(), colors):
    label.set_color(color)  # Set label color
plt.xticks(fontsize=10)
plt.tight_layout()
# Save the figure with the specified DPI
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_x-axis_label.png', dpi=900, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_x-axis_label.pdf', dpi=900, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_x-axis_label.svg', dpi=900, bbox_inches='tight')


# Show the plot
plt.show()


In [ ]:
import pandas as pd
# Assuming cytotoxic_marker is a list or a pandas Series
cytotoxic_marker = pd.Series(['PRF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'FASLG', 'SYTL2', 'CTSW'])  # Example, replace with your actual list

# Check if the genes in cytotoxic_marker are present in the 'names' column of the filtered DataFrame
present_genes = cytotoxic_marker.isin(top_300_per_group[top_300_per_group['cluster_name'] == 'CD8+ Effector T cells']['names'].values)

# Print the result
print(present_genes)


In [ ]:
# Assuming cytotoxic_marker is a list or a pandas Series
proliferating_marker= pd.Series(['MKI67', 'TOP2A', 'GTSE1', 'ASPM', 'KIF4A', 'KIF11', 'KIF14', 'KIF23', 'KNL1', 'CENPF'])  # Example, replace with your actual list

# Check if the genes in cytotoxic_marker are present in the 'names' column of the filtered DataFrame
present_genes = proliferating_marker.isin(top_300_per_group[top_300_per_group['cluster_name'] == "Proliferating CD8+ T cells"]['names'].values)

# Print the result
print(present_genes)


In [ ]:
# Assuming cytotoxic_marker is a list or a pandas Series
treg_marker= pd.Series(['FOXP3', 'IL2RA', 'IL10', 'MAF', 'AREG', 'ICOS', 'IKZF2'])  # Example, replace with your actual list

# Check if the genes in cytotoxic_marker are present in the 'names' column of the filtered DataFrame
present_genes = treg_marker.isin(top_300_per_group[top_300_per_group['cluster_name'] == "CD4+ Regulatory T cells"]['names'].values)

# Print the result
print(present_genes)


In [ ]:
resident_marker =pd.Series(["ITGA1", "ITGAE", "ZNF683",'HOPX','CD69','CD244'])

# Check if the genes in cytotoxic_marker are present in the 'names' column of the filtered DataFrame
present_genes = resident_marker.isin(top_300_per_group[top_300_per_group['cluster_name'] == 'CD8+ Tissue Resident Memory T cells']['names'].values)

# Print the result
print(present_genes)

In [ ]:
Infiltrating_marker = pd.Series(["ITGB2", "ITGAL", 'VCAM1'])
present_genes = Infiltrating_marker.isin(top_300_per_group[top_300_per_group['cluster_name'] == 'CD8+ Effector T cells']['names'].values)
# Print the result
print(present_genes)

In [ ]:
# Assuming cytotoxic_marker and proliferating_marker are lists
cytotoxic_marker = pd.Series(cytotoxic_marker)
proliferating_marker = pd.Series(proliferating_marker)
treg_marker = pd.Series(treg_marker)
Tcell_marker = pd.Series(Tcell_marker)
resident_marker= pd.Series(resident_marker)
Infiltrating_marker= pd.Series(Infiltrating_marker)
Exhaustion_marker = pd.Series(Exhaustion_marker)
CXCR3_marker = pd.Series(CXCR3_marker)
unconventional_marker = pd.Series(unconventional_marker )
Activation_marker = pd.Series(Activation_marker)
# Now you can concatenate them
combined_marker = pd.concat([cytotoxic_marker, proliferating_marker,treg_marker, Tcell_marker, resident_marker, Infiltrating_marker, Exhaustion_marker,CXCR3_marker, unconventional_marker,Activation_marker])

print(combined_marker)

In [ ]:
# Assuming 'cluster_means' and 'genes' are already defined and processed
heat_pre = cluster_means[cluster_means.index.isin(combined_marker)]
z_scores = zscore(heat_pre, axis=1)
sns.set_style("whitegrid", {'axes.grid' : False})
# Generate the clustermap
sns.clustermap(
    z_scores, 
    cmap='viridis', 
    method='average', 
    metric='euclidean', 
    figsize=(6, 40), 
    linewidths=0,  # Set linewidths to 0 to remove grid lines
    row_cluster=True,
    col_cluster=True,
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Assuming 'cluster_means' and 'genes' are already defined and processed
heat_pre = cluster_means[cluster_means.index.isin(genes)]
z_scores = zscore(heat_pre, axis=1)
sns.set_style("whitegrid", {'axes.grid' : False})
# Generate the clustermap
sns.clustermap(
    z_scores, 
    cmap='viridis', 
    method='average', 
    metric='euclidean', 
    figsize=(6, 15), 
    linewidths=0  # Set linewidths to 0 to remove grid lines
)




In [ ]:
# Ran's marker
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/gene_list_for_preliminary_heatmap.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre
z_scores = zscore(heat_pre, axis=1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(6, 15))

In [ ]:
# Elham's marker
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/Pre_marker.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(5, 15), row_cluster=False, col_cluster=False)

In [ ]:
# Shami's marker
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/Chemokine list for heat map.xlsx', header= None)
df[0] = df[0].str.upper()
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(5, 20), row_cluster=False, col_cluster=False)

In [ ]:
#David gene list
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/T-Cell Heatmap Genes-2.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(5, 15), row_cluster=False, col_cluster=False)

In [ ]:
#David gene list
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/David_heat_map_2.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(7,50), row_cluster=False, col_cluster=False)

In [ ]:
#T cell_TF
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/Tcell_TF.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', row_cluster=False, col_cluster=False)

In [ ]:
#Ran list
heat_pre = cluster_means[cluster_means.index.isin(Ran_list)]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=Ran_list)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(7,10), row_cluster=False, col_cluster=False)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['MKI67',
'TOP2A',
'GTSE1',
'ASPM',
'KIF4A',
'KIF11',
'KIF14',
'KIF23',
'KNL1',
'CENPF'], ncols= 3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['SELL',
'CCR7',
'IL7R',
'TCF7',
'LEF1',
'CD27',
'CD28'], ncols= 3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['PRF1',
'GZMA',
'GZMB',
'GZMH',
'GZMK',
'GZMM',
'FASLG',
'SYTL2',
'CTSW'], ncols= 3)


In [ ]:
sc.pl.umap(tcell_ann_concat, color=['CCL5',
'CCR5',
'CXCR3',
'CST7',
'CD69',
'DUSP2',
'KLRG1',
'AOAH'], ncols= 3)

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['LAG3',
'PDCD1',
'CTLA4',
'TIGIT',
'HAVCR2',
'TOX',
'CD101'], ncols= 3)


In [ ]:
sc.pl.umap(tcell_ann_concat, color=['FOXP3',
'IL2RA',
'IL10',
'MAF',
'AREG',
'ICOS'], ncols= 3)


In [ ]:
#LS gene list
df = pd.read_excel('/burg/iicd/users/ls3456/GVHD/adata_file/T-Cell Heatmap Genes_LS.xlsx', header= None)
heat_pre = cluster_means[cluster_means.index.isin(df[0])]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=df[0].values)
heat_pre = heat_pre.T
z_scores = zscore(heat_pre, axis =1)
sns.clustermap(z_scores, cmap='viridis', method='average', metric='euclidean', figsize=(7, 20), row_cluster=False, col_cluster=False)

In [ ]:
heatmap_gene = ['CD3E','CD4','CD8A','CD8B','CCR7', 'SELL','CTLA4', 'FOXP3', 'ICOS','TIGIT','IL2RA','IKZF2','IL2', 'IL23R', 'RORC','IL7R','KLRB1','CD40LG','NFKBIA','GATA3', 'IL13','IL5', 'IL2RA','CD69','TNFRSF4','CD40LG','LTB','ICOS','TRGC2','CD7','XCL1','CAPG','CD160','OSTM1','MKI67','TYMS','MCM7','PDCD1','KLRG1','TIGIT','HAVCR2','LAG3',"GZMA",'GZMB','GZMK','GZMM','PRF1','GNLY','IFNG','PFN1','TYROBP','FCER1G','HLA-B','B2M','CD247']

In [ ]:
heat_pre = cluster_means[cluster_means.index.isin(heatmap_gene)]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=heatmap_gene)
heat_pre = heat_pre.T
heat_pre = zscore(heat_pre, axis = 1)
sns.clustermap(heat_pre, cmap='viridis', method='average', metric='euclidean', figsize=(5, 15))

In [ ]:
heat_pre = cluster_means[cluster_means.index.isin(heatmap_gene)]
heat_pre = heat_pre.T
heat_pre = heat_pre.reindex(columns=heatmap_gene)
heat_pre = heat_pre.T
heat_pre = zscore(heat_pre, axis = 1)
sns.clustermap(heat_pre, cmap='viridis', method='average', metric='euclidean', figsize=(5, 15),row_cluster=False, col_cluster=False)

In [ ]:
tcell_ann_concat.obs['PatientID_Grade'].unique()

In [ ]:
tcell_ann_concat.obs[tcell_ann_concat.obs['PatientID']=='C160']

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['PatientID_Grade'])

In [ ]:
tcell_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_concat_Tcell_only_proportion.h5ad'
tcell_ann_concat.write(tcell_ann_loc)

# Cell type proportion analysis

In [ ]:
tcell_ann_concat = anndata.read('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/tcell_ann_concat_Tcell_only_proportion.h5ad')

In [ ]:
tcell_ann_concat

In [ ]:
dedf = pd.read_csv('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/dedf_Tcells_leiden_0.5_w_notraw.csv')
dedf

In [ ]:
sc.pl.umap(tcell_ann_concat, color = ['LEF1','GK','PASK','AP3M2', 'RAMP2'])

In [ ]:
# COEXPRESSED IN CD3+ spots
sc.pl.umap(tcell_ann_concat, color = ['RAMP2', "CA9",'KHK','OLFM4','HDC','HPGDS','SIGLEC6','EPCAM','MUC5AC','MUC6','PGC','TFF1','TFF2'])


In [ ]:
# COEXPRESSED IN CD3+ spots
sc.pl.umap(tcell_ann_concat, color = ['ALDOB','DRD5', 'GUCA2B', 'HOPX', 'LIPF'])

In [ ]:
sc.pl.umap(tcell_ann_concat, color = ['ASCL2', 'CENPM', 'EPHB2', 'LGR5', 'MSI1', 'OLFM4', 'RARRES2', 'SMOC2', 'SOX9'])

In [ ]:
sc.pl.umap(tcell_ann_concat, color = ['CD320','JAML'])

In [ ]:
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'viridis'  # Or your preferred colormap
mpl.rcParams.update({'font.size': 10})
mpl.rcParams['axes.titlesize'] = 10
mpl.rcParams['xtick.labelsize'] = 10  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 10  # Colorbar tick label size
fig = sc.pl.umap(tcell_ann_concat, 
           color=['MUC5AC', 'MUC6', 'PGC', 'TFF1', 'TFF2', 'CLDN18', 'CTSE', 'GKN2', 'MUCL3', 'SIGLEC6'],
           ncols=5,
          show = False,
          frameon=False,
          vmin = 0,
          vmax = 2,
          return_fig = True,
                size = 1)

fig.set_size_inches(7.5, 3)

plt.tight_layout()

# Save the figure at 300 DPI
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_epi_overlap.png", dpi=300, bbox_inches='tight')
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_epi_overlap.pdf", dpi=300, bbox_inches='tight')
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_epi_overlap.svg", dpi=300, bbox_inches='tight')
##ptionally, show the plot after saving
plt.show()

In [ ]:
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'viridis'  # Or your preferred colormap
mpl.rcParams.update({'font.size': 10})
mpl.rcParams['axes.titlesize'] = 10
mpl.rcParams['xtick.labelsize'] = 10  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 10  # Colorbar tick label size
fig = sc.pl.umap(tcell_ann_concat, 
           color=['OLFM4','ANPEP','EPCAM','CEACAM5','SNORC','TFF3'],
           ncols=3,
          show = False,
          frameon=False,
          vmin = 0,
          vmax = 2,
          return_fig = True,
                size = 1)

fig.set_size_inches(3.75, 3)

plt.tight_layout()

# Save the figure at 300 DPI
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_ins_epi_overlap.png", dpi=300, bbox_inches='tight')
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_ins_epi_overlap.pdf", dpi=300, bbox_inches='tight')
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/spatial_figures/Tcell_ins_epi_overlap.svg", dpi=300, bbox_inches='tight')
##ptionally, show the plot after saving
plt.show()

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

# For each gene, count cells with expression > 0
genes_of_interest = ['RAMP2', 'CA9', 'KHK', 'OLFM4', 'HDC', 'HPGDS', 'SIGLEC6','EPCAM','EPCAM', 'OLFM4','IGLC1','TXNDC5','CEACAM5','TFF3','MUC5AC','MUC6','PGC','TFF1','TFF2']
genes_of_interest= ['MUC5AC','MUC6','PGC','TFF1','TFF2', 'CLDN18','CTSE','GKN2','MUCL3','SIGLEC6']
expression_counts = {}

for gene in genes_of_interest:
    # Check if gene exists in dataset
    if gene in tcell_ann_concat.var_names:
        # Count cells where expression > 0
        expr_values = tcell_ann_concat[:, gene].X
        
        # Handle sparse matrix if needed
        if isinstance(expr_values, np.ndarray):
            positive_cells = np.sum(expr_values > 0)
        else:  # For sparse matrix
            positive_cells = expr_values.toarray().flatten()
            positive_cells = np.sum(positive_cells > 0)
            
        # Calculate percentage
        percent = (positive_cells / tcell_ann_concat.n_obs) * 100
        
        # Store results
        expression_counts[gene] = {
            'cells_expressing': int(positive_cells),
            'percent_expressing': round(percent, 2)
        }
    else:
        expression_counts[gene] = {'cells_expressing': 'Gene not found', 'percent_expressing': 'N/A'}

# Convert to DataFrame for better visualization
results_df = pd.DataFrame(expression_counts).T
print(results_df)

In [ ]:
print(list(dedf[dedf['group'] ==6]['names'][300:400]))

In [ ]:
for patient in tcell_ann_concat.obs['PatientID'].unique():
    tcell_ann_patient= tcell_ann_concat[tcell_ann_concat.obs['PatientID']== patient]
    print(patient)
    sc.pl.umap(tcell_ann_patient, color=['ZNF683'])

# Figure 4D

In [ ]:
sorter = ['ND001_ND','ND003_ND','C66_No','R085_No','C162_No','C160_No','C155_No','C160_Mild','C157_Mild','C179_Mild','C101_Mild','C162_Mild','C159_Severe','C152_Severe','C068_Severe','C98_Severe','C66_Severe']

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"CD8+ Proliferating T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells'
]
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)
from copy import deepcopy
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3),  # Set default figure size to 3×3 inches
    'lines.linewidth': 1  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size
obs_1 = 'PatientID_Grade'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'], edgecolor='none')
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left', frameon=False)
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cell_type_bar_plot.png', dpi=300, bbox_inches='tight', transparent = True)  # Save PNG
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cell_type_bar_plot.pdf', dpi=300, bbox_inches='tight', transparent = True)  # Save PDF
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cell_type_bar_plot.svg', dpi=300, bbox_inches='tight', transparent = True)  # Save PDF
plt.show()


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=tcell_ann_concat.uns[obs_1 + '_colors'],edgecolor='none')
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cell_type_bar_plot_2.png', dpi=300, bbox_inches='tight')  # Save PNG
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cell_type_bar_plot_2.pdf', dpi=300, bbox_inches='tight')  # Save PDF
plt.show()

In [ ]:
from copy import deepcopy
#savepath = '/Users/lingtingshi/Documents/Azizi_lab/GVHD/'
obs_1 = 'PatientID_Grade'
obs_2 = 'leiden_48'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True)
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True)
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import rcParams
import seaborn as sns
import statannotations
sns.set_style('white')

font_list = []
fpaths = matplotlib.font_manager.findSystemFonts()
for i in fpaths:
    try:
        f = matplotlib.font_manager.get_font(i)
        font_list.append(f.family_name)
    except RuntimeError:
        pass

font_list = set(font_list)
plot_font = 'Arial'

rcParams['font.family'] = plot_font
rcParams.update({'font.size': 15})
rcParams.update({'figure.dpi': 300})
rcParams.update({'figure.figsize': (4, 4)})
rcParams.update({'savefig.dpi': 300})

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install statannotations==0.5.0

In [ ]:
#!pip install seaborn==0.11.1

In [ ]:
#!conda activate scanpy_c

In [ ]:
#!pip list

In [ ]:
#!conda info --envs

In [ ]:
from statannotations.Annotator import Annotator
# Assuming adata is already defined and contains the necessary data
obs_1 = 'PatientID_Grade'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters
    df_obs2 = df_obs2.reindex(columns=sorter)
    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num = num + 1
    p_list = list()

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_mild,df_severe,])
        rcParams.update({'font.size': 10})
        fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()

        plt.show()


In [ ]:
tcell_ann_concat.obs['SampleID_Grade'] = tcell_ann_concat.obs['SampleID'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)

In [ ]:
sc.pl.umap(tcell_ann_concat, color='SampleID_Grade')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
from copy import deepcopy
from statannotations.Annotator import Annotator

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
        rcParams.update({'font.size': 10})
        fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()

        plt.show()

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident \nMemory T cells', "CD4+ \nCentral Memory T cells","CD8+ Cytotoxic \nUnconventional T cells",'Low CD3',"Proliferating \nCD8+ T cells","CD4+ \nRegulatory T cells",'CD8+ Homeostatic \nUnconventional T cells ']
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
from copy import deepcopy
from statannotations.Annotator import Annotator

# Assuming adata is already defined and contains the necessary data
obs_1 = 'GVHD_Grade_1'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat]:
    adata.obs[obs_1] = adata.obs[obs_1].replace('ND', 'No')
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
        rcParams.update({'font.size': 10})
        fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()

        plt.show()

In [ ]:
def create_radar_plot(df, title="T Cell Subset Distribution"):
    # Define the desired order of cell types
    ordered_cell_types = [
        'CD4+ \nCentral Memory T cells',
        'CD4+ \nRegulatory T cells',
        'CD4+ Effector T cells',
        'CD8+ Tissue Resident \nMemory T cells',
        'CD8+ Effector T cells',
        'Proliferating \nCD8+ T cells',
        'CD8+ Homeostatic \nUnconventional T cells ',
        'CD8+ Cytotoxic \nUnconventional T cells',
        'Low CD3'
    ]
    
    # Define the desired order of GVHD grades
    grade_order = ['No', 'Mild', 'Severe']
    
    # Reorder the DataFrame
    df = df.reindex(ordered_cell_types)
    df = df[grade_order]  # Reorder columns according to grade order
    
    categories = df.index.tolist()
    num_vars = len(categories)
    angles = [n / float(num_vars) * 2 * np.pi for n in range(num_vars)]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(5.5, 5.5), subplot_kw=dict(projection='polar'))
    
    colors = {'No': '#09BB8C', 'Mild': '#005A8F', 'Severe': '#B85000'}
    alphas = {'No': 0.25, 'Mild': 0.25, 'Severe': 0.25}
    
    for column in grade_order:  # Use grade_order to ensure correct plotting order
        values = df[column].values.tolist()
        print(f"Sum of {column} before closing the polygon: {np.sum(values)}")
        values += values[:1]
        ax.plot(angles, values, 'o-', linewidth=2, label=column, 
                color=colors[column])
        ax.fill(angles, values, alpha=alphas[column], color=colors[column])
    
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.grid(True)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.legend(loc='upper right', bbox_to_anchor=(0, 0.1))
    
    plt.tight_layout()
    return fig

# Create and save the plot
fig = create_radar_plot(df_obs2)
plt.tight_layout()
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/radar_plot_mean.pdf', bbox_inches='tight', dpi=300)
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/radar_plot_mean.svg', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
df = pd.DataFrame({
    'cell_type': tcell_ann_concat.obs['leiden_0.5'],
    'grade': tcell_ann_concat.obs['GVHD_Grade_1'],
    'sample': tcell_ann_concat.obs['SampleID']
})

# Calculate percentages per sample
sample_percentages = pd.crosstab(
    [df['sample'], df['grade']], 
    df['cell_type'], 
    normalize='index'
) * 100

In [ ]:
median_percentages = sample_percentages.groupby(level='grade').median()
median_percentages.T
df_obs2 = median_percentages.T
df_obs2

In [ ]:
normalized_data = df_obs2 / df_obs2.sum(axis=0)
df_obs2 = normalized_data 

# Figure 4G

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'svg.fonttype': 'none',
    'font.family': 'Arial',
})

mpl.rcParams.update({
    'font.size': 10,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
})

mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size
mpl.rcParams['legend.fontsize'] = 8     # For legend text

def create_radar_plot(df, title="T Cell Subset Distribution"):
    # Define the desired order of cell types
    ordered_cell_types = [
        'CD4+ \nCentral Memory T cells',
        'CD4+ \nRegulatory T cells',
        'CD4+ Effector T cells',
        'CD8+ Tissue Resident \nMemory T cells',
        'CD8+ Effector T cells',
        'Proliferating \nCD8+ T cells',
        'CD8+ Homeostatic \nUnconventional T cells ',
        'CD8+ Cytotoxic \nUnconventional T cells',
        'Low CD3'
    ]

    grade_order = ['No', 'Mild', 'Severe']
    
    # Reorder the DataFrame
    df = df.reindex(ordered_cell_types)
    df = df[grade_order]  # Reorder columns according to grade order
    
    # Rest of your original code remains the same
    categories = df.index.tolist()
    num_vars = len(categories)
    angles = [n / float(num_vars) * 2 * np.pi for n in range(num_vars)]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(projection='polar'))
    
    colors = {'No': '#09BB8C', 'Mild': '#005A8F', 'Severe': '#B85000'}
    alphas = {'No': 0.25, 'Mild': 0.25, 'Severe': 0.25}
    
    for column in df.columns:
        values = df[column].values.tolist()
        print(f"Sum of {column} before closing the polygon: {np.sum(values)}")
        values += values[:1]
        ax.plot(angles, values, 'o-', linewidth=2, label=column, 
                color=colors[column])
        ax.fill(angles, values, alpha=alphas[column], color=colors[column])
    
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.grid(True)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.legend(loc='upper right', bbox_to_anchor=(0, 0.1))
    
    #plt.title(title, y=1.05)
    return fig

# Create and save the plot
fig = create_radar_plot(df_obs2)
plt.tight_layout()
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/radar_plot_median.pdf', bbox_inches='tight', dpi=300, transparent = True)
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/radar_plot_median.png', bbox_inches='tight', dpi=300, transparent = True)
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/radar_plot_median.svg', bbox_inches='tight', dpi=300, transparent = True)
plt.show()

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"CD8+ Proliferating T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells ']
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)
cd8_columns = ['CD8+ Effector T cells','CD8+ Tissue Resident Memory T cells','CD8+ Cytotoxic Unconventional T cells','CD8+ Proliferating T cells','CD8+ Homeostatic Unconventional T cells ','Low CD3']
cd4_columns = ['CD4+ Regulatory T cells', 'CD4+ Effector T cells','CD4+ Central Memory T cells',]

In [ ]:
tcell_ann_concat.obs['leiden_48'] = tcell_ann_concat.obs['leiden_0.5'].apply(
    lambda x: 'CD4' if x in cd4_columns else 'CD8'
)

tcell_ann_concat.obs['leiden_48'] = tcell_ann_concat.obs['leiden_48'].astype('category')

In [ ]:
ptcy = ['C155','C101','C162','C160']

tcell_ann_concat.obs['ptcy'] = 'NoP'
tcell_ann_concat.obs.loc[tcell_ann_concat.obs['PatientID'].isin(ptcy),'ptcy'] = "Ptcy"

tcell_ann_concat.obs['PatientID_Grade_Ptcy'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat.obs['ptcy'].astype(str)

tcell_ann_concat.obs['PatientID_Grade_Ptcy']=tcell_ann_concat.obs['PatientID_Grade_Ptcy'].astype("category")

tcell_ann_concat.obs['SampleID_Ptcy'] = tcell_ann_concat.obs['SampleID'].astype(str)+'_'+tcell_ann_concat.obs['ptcy'].astype(str)

tcell_ann_concat.obs['SampleID_Ptcy']=tcell_ann_concat.obs['SampleID_Ptcy'].astype("category")

tcell_ann_concat.obs['SampleID_Tissue_Location'] = tcell_ann_concat.obs['SampleID'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)

tcell_ann_concat.obs['SampleID_Tissue_Location']=tcell_ann_concat.obs['SampleID_Tissue_Location'].astype("category")

tcell_ann_concat_Upper = tcell_ann_concat[tcell_ann_concat.obs['Tissue_Location'] == 'Upper GI']

tcell_ann_concat_Upper_LP = tcell_ann_concat_Upper[tcell_ann_concat_Upper.obs['LPorIE'] == 'Lamina Propria Cells']
tcell_ann_concat_Upper_IE = tcell_ann_concat_Upper[tcell_ann_concat_Upper.obs['LPorIE'] == 'Intraepithelial Cells']

tcell_ann_concat_Lower = tcell_ann_concat[tcell_ann_concat.obs['Tissue_Location'] == 'Lower GI']
tcell_ann_concat_Lower_LP = tcell_ann_concat_Lower[tcell_ann_concat_Lower.obs['LPorIE'] == 'Lamina Propria Cells']
tcell_ann_concat_Lower_IE = tcell_ann_concat_Lower[tcell_ann_concat_Lower.obs['LPorIE'] == 'Intraepithelial Cells']

In [ ]:
sc.pl.umap(tcell_ann_concat, color = ['SampleID_Ptcy'])
sc.pl.umap(tcell_ann_concat, color = ['SampleID_Tissue_Location'])


In [ ]:
###### Filter out samples with low cell count
# Calculate cell counts per sample
cell_counts = tcell_ann_concat.obs['SampleID'].value_counts()

# Get samples with at least 100 cells
valid_samples = cell_counts[cell_counts >= 100].index

# Subset AnnData to retain only valid samples
tcell_ann_concat_f = tcell_ann_concat[tcell_ann_concat.obs['SampleID'].isin(valid_samples), :]

# Verify filtering
print(f"Remaining samples: {tcell_ann_concat_f.obs['SampleID'].nunique()}")
print(f"Remaining cells: {tcell_ann_concat_f.n_obs}")

In [ ]:
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator  # Assuming you use statannotations for the p-values

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_48'
num = 1

for adata in [tcell_ann_concat_f]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs[obs_2].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
        print(result)
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(3, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'],showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.png', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.pdf', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.svg', dpi=300, transparent = True)
        


        plt.show()

In [ ]:
CD48_df = df_obs2.T

In [ ]:
CD48_df['CD4/CD8'] = CD48_df['CD4']/CD48_df['CD8']
df_obs2 = CD48_df.T
CD48_df

In [ ]:
i = 'CD4/CD8'
substring = 'Severe'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_severe = df_obs2.loc[:, column_mask]
severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

substring = 'Mild'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_mild = df_obs2.loc[:, column_mask]
mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

substring = 'No'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_no = df_obs2.loc[:, column_mask]
no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

substring = 'Treated'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_treated = df_obs2.loc[:, column_mask]
treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


substring = 'ND'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_nd = df_obs2.loc[:, column_mask]
nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]


df_severe = pd.DataFrame()
df_severe['Value'] = severe
df_severe['Grade'] = 'severe'

df_mild = pd.DataFrame()
df_mild['Value'] = mild
df_mild['Grade'] = 'mild'

df_no = pd.DataFrame()
df_no['Value'] = no
df_no['Grade'] = 'no'

df_treated = pd.DataFrame()
df_treated['Value'] = treated
df_treated['Grade'] = 'no'

df_ND = pd.DataFrame()
df_ND['Value'] = nd
df_ND['Grade'] = 'no'


palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
print(result)
rcParams.update({'font.size': 14})
fig, ax = plt.subplots(figsize=(3, 2), dpi=300)
sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'],showfliers=False)
sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

ax.spines[['right', 'top']].set_visible(False)
ax.set_xlabel('')
ax.set_ylabel('Cell type proportion')
ax.set_title(i)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
#annotator.configure(test='Kruskal')
annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
annotator.apply_and_annotate()
plt.tight_layout()

# Save figure with 300 DPI
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd4Ncd8_boxplot_300dpi.png', dpi=300, transparent = True)
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd4Ncd8_boxplot_300dpi.pdf', dpi=300, transparent = True)
plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd4Ncd8_boxplot_300dpi.svg', dpi=300, transparent = True)

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"CD8+ Proliferating T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells ']
tcell_ann_concat_f.rename_categories('leiden_0.5', new_cluster_names)

In [ ]:
import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1.3 
mpl.rcParams['axes.grid'] = False  # Disable grid lines

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Tissue_Location'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat_f]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Upper GI'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Lower GI'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'Upper GI'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'Lower GI'
        

        palette = {'Lower GI': '#09BB8C', 'Upper GI': '#005A8F'}

        result = pd.concat([df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['Upper GI','Lower GI'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['Upper GI','Lower GI'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('Upper GI','Lower GI')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney')
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_tissue{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_tissue{i}_boxplot_300dpi.pdf', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_tissue{i}_boxplot_300dpi.svg', dpi=300)

        plt.show()

In [ ]:
import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1.3 
mpl.rcParams['axes.grid'] = False  # Disable grid lines

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Ptcy'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat_f]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Ptcy'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'NoP'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'Ptcy'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'NoP'
        

        palette = {'NoP': '#09BB8C', 'Ptcy': '#005A8F'}

        result = pd.concat([df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['Ptcy','NoP'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['Ptcy','NoP'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('Ptcy','NoP')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney')
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.pdf', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.svg', dpi=300)

        plt.show()

# Figure 4

In [ ]:
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator  # Assuming you use statannotations for the p-values

import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1 
mpl.rcParams['axes.grid'] = False  # Disable grid lines
# Set all text elements to black color
mpl.rcParams['text.color'] = 'black'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['axes.titlecolor'] = 'black'
mpl.rcParams['legend.facecolor'] = 'white'
mpl.rcParams['legend.edgecolor'] = 'black'
mpl.rcParams['legend.labelcolor'] = 'black'
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
})

mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1

for adata in [tcell_ann_concat_f]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
        fig, ax = plt.subplots(figsize=(2.5, 1.3), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportions')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.png', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.pdf', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{i}_boxplot_300dpi.svg', dpi=300, transparent = True)

        plt.show()

In [ ]:
df_obs2_T= df_obs2.T
df_obs2_T.index

In [ ]:
df_obs2

In [ ]:
df_Treg_1= (df_obs2.T['CD4+ Regulatory T cells']/df_obs2.T['CD8+ Effector T cells']).T
df_Treg_1.index

In [ ]:
# Treg normalized to cd8+ Effector population
substring = 'Severe'
column_mask = df_Treg_1.index.str.contains(substring)
# Use the mask to select the desired columns
selected_columns_severe = df_Treg_1.loc[column_mask,]
severe = selected_columns_severe
substring = 'Mild'
column_mask = df_Treg_1.index.str.contains(substring)
# Use the mask to select the desired columns
selected_columns_mild = df_Treg_1.loc[column_mask,]
mild = selected_columns_mild
        
t_stat, p_value = stats.ttest_ind(severe, mild)
p_list.append(p_value)
print(i)
print(p_value)

bp = plt.boxplot([mild, severe], labels=['Mild', 'Severe'], patch_artist=True)

# Set the colors of the boxes
colors = ['#F6F740', '#D64933']
plt.title('Treg normalized to cd8+ Effector population', fontsize=14)


### compare to No

In [ ]:


df_Treg_2 = (df_obs2.T['CD4+ Regulatory T cells']/df_obs2.T[cd8_columns].sum(axis = 1)).T
df_4_8_2 = (df_obs2.T[cd8_columns].sum(axis = 1)/df_obs2.T[cd4_columns].sum(axis = 1)).T


In [ ]:
# Treg normalized to all CD8+ T cells
p_list = []
substring = 'Severe'
column_mask = df_Treg_2.index.str.contains(substring)
# Use the mask to select the desired columns
selected_columns_severe = df_Treg_2.loc[column_mask,]
severe = selected_columns_severe
substring = 'Mild'
column_mask = df_Treg_2.index.str.contains(substring)
# Use the mask to select the desired columns
selected_columns_mild = df_Treg_2.loc[column_mask,]
mild = selected_columns_mild
        
t_stat, p_value = stats.ttest_ind(severe, mild)
p_list.append(p_value)
print(p_value)

bp = plt.boxplot([mild, severe], labels=['Mild', 'Severe'], patch_artist=True)

# Set the colors of the boxes
colors = ['#F6F740', '#D64933']
plt.title('Treg normalized to all CD8+ T cells', fontsize=14)

### compare to No

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

p_list = []

# Filter for severe cases
substring_severe = 'Severe'
column_mask_severe = df_4_8_2.index.str.contains(substring_severe)
selected_columns_severe = df_4_8_2.loc[column_mask_severe]
severe = selected_columns_severe.values.flatten()

# Filter for mild cases
substring_mild = 'Mild'
column_mask_mild = df_4_8_2.index.str.contains(substring_mild)
selected_columns_mild = df_4_8_2.loc[column_mask_mild]
mild = selected_columns_mild.values.flatten()

# Perform t-test
t_stat, p_value = stats.ttest_ind(severe, mild)
p_list.append(p_value)
print('P-value:', p_value)

# Create boxplot
bp = plt.boxplot([mild, severe], labels=['No/ND', 'Mild/Severe'], patch_artist=True)

# Set the colors of the boxes
colors = ['#F6F740', '#D64933']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

# Add individual dots
positions = [1, 2]
for i, data in enumerate([mild, severe]):
    x = [positions[i]] * len(data)
    plt.scatter(x, data, alpha=0.6, edgecolors='k', zorder=2)

plt.title('CD8/CD4', fontsize=14)
plt.show()



In [ ]:
zscore(df_obs2)

In [ ]:
sns.clustermap(zscore(df_obs2), cmap='viridis', method='average', metric='euclidean', figsize=(15, 6))

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
linkage_matrix = linkage(zscore(df_obs2).T, metric='chebyshev')
plt.subplots(figsize=(24, 6))  # Adjust the values (width, height) as needed
dendrogram(linkage_matrix, labels = df_obs2_T.index, truncate_mode= 'level')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.title('Hierarchical Clustering Dendrogram')
plt.show()

In [ ]:
tcell_ann_concat.obs

In [ ]:
#!pip install statannotations

In [ ]:
#!pip install scikit_posthocs

In [ ]:
# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1

ann_data = tcell_ann_concat_f[tcell_ann_concat_f.obs['leiden_0.5'].isin(cd4_columns)]

for adata in [ann_data]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe])
        rcParams.update({'font.size': 8})
        fig, ax = plt.subplots(figsize=(3.2, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd4_{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd4_{i}_boxplot_300dpi.pdf', dpi=300)

        plt.show()

In [ ]:
# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1

ann_data = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'].isin(cd8_columns)]

for adata in [ann_data]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe ])
        rcParams.update({'font.size': 10})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()

        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd8_{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_cd8_{i}_boxplot_300dpi.pdf', dpi=300)

        plt.show()


        

In [ ]:
# Assuming df_obs2 is a DataFrame with the appropriate rows and columns
result_df = (df_obs2.loc['CD8+ Effector T cells'] / df_obs2.loc['CD8+ Tissue Resident Memory T cells']).to_frame().T

# Set the original columns back
result_df.columns = df_obs2.columns

# Display the resulting DataFrame
print(result_df)


In [ ]:
substring = 'ND'
column_mask = df_obs2.columns.str.contains(substring)
selcted_columns_nd = df_obs2.loc[:, column_mask]

In [ ]:
nd = selected_columns_nd.values[~np.isnan(selected_columns_nd.values)]

In [ ]:
substring = 'Severe'
df_obs2 = result_df
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_severe = df_obs2.loc[:, column_mask]
severe = selected_columns_severe.values[~np.isnan(selected_columns_severe.values)]
print(severe)

substring = 'Mild'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_mild = df_obs2.loc[:, column_mask]
mild = selected_columns_mild.values[~np.isnan(selected_columns_mild.values)]
print(mild)

substring = 'No'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_no = df_obs2.loc[:, column_mask]
no = selected_columns_no.values[~np.isnan(selected_columns_no.values)]
print(no)
substring = 'Treated'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_treated = df_obs2.loc[:, column_mask]
treated = selected_columns_treated.values[~np.isnan(selected_columns_treated.values)]
print(treated)

substring = 'ND'
column_mask = df_obs2.columns.str.contains(substring)
selected_columns_nd = df_obs2.loc[:, column_mask]
nd = selected_columns_nd.values[~np.isnan(selected_columns_nd.values)]
print(nd)
        
df_severe = pd.DataFrame()
df_severe['Value'] = severe
df_severe['Grade'] = 'severe'

df_mild = pd.DataFrame()
df_mild['Value'] = mild
df_mild['Grade'] = 'mild'

df_no = pd.DataFrame()
df_no['Value'] = no
df_no['Grade'] = 'no'

df_treated = pd.DataFrame()
df_treated['Value'] = treated
df_treated['Grade'] = 'no'

df_ND = pd.DataFrame()
df_ND['Value'] = nd
df_ND['Grade'] = 'no'
        

palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe ])
rcParams.update({'font.size': 10})
fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

ax.spines[['right', 'top']].set_visible(False)
ax.set_xlabel('Grade')
ax.set_ylabel('Cell type proportion')
ax.set_title("Effector CD8+ T cell / Resident CD8+ T cell ")
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
#annotator.configure(test='Kruskal')
annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
annotator.apply_and_annotate()

plt.show()

In [ ]:
selected_columns_severe

In [ ]:
#!pip install statsmodels

In [ ]:
tcell_ann_concat.obs['LPorIE']

In [ ]:
# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_48'
num = 1

LP = tcell_ann_concat_f[tcell_ann_concat_f.obs['LPorIE']=="Lamina Propria Cells" ]
IE = tcell_ann_concat_f[tcell_ann_concat_f.obs['LPorIE']=='Intraepithelial Cells' ]
for adata in [LP,IE]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_48'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe])
        rcParams.update({'font.size': 10})
        fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'])
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        #annotator.configure(test='Kruskal')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()

        plt.show()

# LP vs IE CELL TYPE Proportion

In [ ]:
tcell_ann_concat_f.obs

In [ ]:
# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1

for region in ["Lamina Propria Cells", 'Intraepithelial Cells']:

    adata = tcell_ann_concat_f[tcell_ann_concat_f.obs['LPorIE']==region ]
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2.5), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()
        plt.tight_layout()


        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{region}_{i}_boxplot_300dpi.png', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{region}_{i}_boxplot_300dpi.pdf', dpi=300, transparent = True)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_{region}_{i}_boxplot_300dpi.svg', dpi=300, transparent = True)

        plt.show()

In [ ]:
# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_0.5'
num = 1
# Define CD4+ T cell subsets of interest
cd4_subsets = ['CD4+ Effector T cells', 'CD4+ Regulatory T cells', 'CD4+ Central Memory T cells']


for region in ["Lamina Propria Cells", 'Intraepithelial Cells']:

    adata = tcell_ann_concat_f[tcell_ann_concat_f.obs['LPorIE']==region ]
    adata = adata[adata.obs['leiden_0.5'].isin(cd4_subsets)]
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND, df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2.5), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('Grade')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        annotator.apply_and_annotate()
        plt.tight_layout()


        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cd4_fig_{region}_{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/cd4_fig_{region}_{i}_boxplot_300dpi.pdf', dpi=300)

        plt.show()

In [ ]:
tcell_ann_loc = '/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/tcell_ann_concat_proportion_only.h5ad'
tcell_ann_concat_f.write(tcell_ann_loc)

# CD4 and CD8 and Treg

In [ ]:
sc.pl.violin(tcell_ann_concat, ['ZBTB7B','RUNX3','CD4','CD8A','CD8B'], groupby='leiden_48')

In [ ]:
sc.tl.rank_genes_groups(tcell_ann_concat, groupby='leiden_48', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(tcell_ann_concat, group="Double_Pos")
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]


crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(6,6), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=3)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Double Pos Versus Other T cells', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

# Tregs

In [ ]:
new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"CD8+ Proliferating T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells ']
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)
Treg_cells= tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5']=='CD4+ Regulatory T cells']
Treg_cells

In [ ]:
Treg_cell_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/Treg_ann.h5ad'
Treg_cells.write(Treg_cell_loc)

In [ ]:
tcell_ann_concat_100 = tcell_ann_concat[tcell_ann_concat.obs['Days_post-Transplant']<=100]

In [ ]:
Treg_cells_100= tcell_ann_concat_100[tcell_ann_concat_100.obs['leiden_0.5']=='CD4+ Regulatory T cells']

In [ ]:
Treg_cells_100

In [ ]:
Treg_100_cell_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/Treg__100ann.h5ad'
Treg_cells_100.write(Treg_100_cell_loc)

In [ ]:
sc.pl.umap(Treg_cells_100, color='ptcy')

In [ ]:
import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1.3 
mpl.rcParams['axes.grid'] = False  # Disable grid lines

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Ptcy'
obs_2 = 'leiden_0.5'
num = 1

tcell_ann_concat_100 = tcell_ann_concat_f[tcell_ann_concat_f.obs['Days_post-Transplant']<=100]

for adata in [tcell_ann_concat_100]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Ptcy'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'NoP'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'Ptcy'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'NoP'
        

        palette = {'NoP': '#09BB8C', 'Ptcy': '#005A8F'}

        result = pd.concat([df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['Ptcy','NoP'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['Ptcy','NoP'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('Ptcy','NoP')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney')
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.png', dpi=300)
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.pdf', dpi=300)
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.svg', dpi=300)

        plt.show()

In [ ]:
import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1.3 
mpl.rcParams['axes.grid'] = False  # Disable grid lines

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Ptcy'
obs_2 = 'leiden_0.5'
num = 1

tcell_ann_concat_100 = tcell_ann_concat_f[tcell_ann_concat_f.obs['Days_post-Transplant']<=100]
tcell_ann_concat_100 = tcell_ann_concat_100[tcell_ann_concat_100.obs['leiden_0.5'].isin(cd4_subsets)]
for adata in [tcell_ann_concat_100]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Ptcy'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'NoP'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'Ptcy'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'NoP'
        

        palette = {'NoP': '#09BB8C', 'Ptcy': '#005A8F'}

        result = pd.concat([df_mild, df_severe])
        rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['Ptcy','NoP'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['Ptcy','NoP'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('Ptcy','NoP')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney')
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.png', dpi=300)
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.pdf', dpi=300)
        #plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/fig_ptcy{i}_boxplot_300dpi.svg', dpi=300)

        plt.show()

In [ ]:
Treg_cells = anndata.read_h5ad('/burg/iicd/users/ls3456/GVHD/adata_file/Treg_ann.h5ad')

In [ ]:
sc.pl.umap(Treg_cells, color='GVHD_Grade_1')

In [ ]:
# Severe vs rest
Treg_cells.obs['Grade_Svsall'] = "Rest"
Treg_cells.obs.loc[Treg_cells.obs['GVHD_Grade_1']== "Severe",'Grade_Svsall']= "Severe"

In [ ]:
sc.pl.umap(Treg_cells, color='Grade_Svsall')

In [ ]:
sc.tl.rank_genes_groups(Treg_cells, groupby='Grade_Svsall', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(Treg_cells, group="Severe")
degs_df.to_csv('/Users/lingting/Documents/GVHD_project/Treg_SeverevsRest.csv')

In [ ]:
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]

In [ ]:
Treg_sig = pd.read_excel('/Users/lingting/Documents/GVHD_project/Treg_cytotoxic_gene_sig.xlsx')

In [ ]:
gene_categories = {
    "Canonical_Treg_genes": ["FOXP3", "IKZF2", "IL2R", "CTLA4"],
    
    "OxPhos": ["COX4I1", "COX5B", "COX6A1", "COX6B1", "COX7A2", "COX7C", 
               "NDUFS5", "UQCRB", "ATP5F1E", "ATP5MG", "ATP5MC2"],
    
    "TCR_components": ["CD3E", "TRBC2"],
    
    "MHC_components": ["HLA-A", "HLA-B", "HLA-C", "B2M", "HLA-DQB1", "CD74"],
    
    "Cytokine_receptors": ["IL1R1", "IL18R1", "IL6ST", "IL2RA", "IL2RB", "IL2RG"],
    
    "Interferon_stimulated_genes": ["GBP1", "GBP2", "IRF1", "IRF9"],
    
    "Chemokine_receptors": ["CCR4", "CCR6", "CXCR6", "CCR8"],
    
    "JAK_STAT": ["JAK1", "STAT1", "STAT3", "STAT5"],
    
    "Costimulatory": ["LAG3", "ICOS", "TNFRSF9", "TNFRSF1B"],
    
    "Other_transcription_factors": ["IKZF1", "ID2", "FOXO1", "KLF13"],
    "Cytotoxic Treg": ['PRF1','GZMA','GZMB','CCL4','IL2RA','IL2RG','IL2RB','FOXP3','GBP2', 'GBP5','IL6ST','STAT3','JAK1','LAG3','IKZF1','CTLA4']
    
}

# If you want a flat list of all genes
all_genes = [gene for category in gene_categories.values() for gene in category]

# Print all genes
print(all_genes)

In [ ]:
def plot_marker_heatmap(adata, gene_list, groupby='Severe', figsize=(2, 1.5)):
    """
    Plot a heatmap of z-scored expression values for marker genes.
    
    Parameters:
    -----------
    adata : AnnData
        The annotated data matrix.
    gene_list : list
        List of gene names to include in the heatmap.
    groupby : str
        Column in adata.obs to group cells by.
    figsize : tuple
        Figure size (width, height).
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        The figure object containing the heatmap.
    """
    # Check if all genes are in the dataset
    missing_genes = [gene for gene in gene_list if gene not in adata.var_names]
    if missing_genes:
        print(f"Warning: The following genes are not in the dataset: {missing_genes}")
        gene_list = [gene for gene in gene_list if gene in adata.var_names]
    
    if len(gene_list) == 0:
        print("No valid genes to plot!")
        return None
    
    # Create a copy of the AnnData object with only the genes of interest
    adata_subset = adata[:, gene_list].copy()
    
    # Z-score the data (scale each gene)
    sc.pp.scale(adata_subset)
    print("Min value:", np.min(adata_subset.X))
    print("Max value:", np.max(adata_subset.X))
    print("Negative values:", np.sum(adata_subset.X < 0))
        
    # Get average expression by group
    if groupby in adata_subset.obs.columns:
        # Clear current figure if any
        plt.clf()
        
        # First, compute dendrogram
        #sc.tl.dendrogram(adata_subset, groupby=groupby)
        
        # Now create the heatmap
        sc.pl.heatmap(adata_subset, 
                      var_names=gene_list, 
                      groupby=groupby,
                      use_raw = False,
                      dendrogram=False,
                      swap_axes=True,
                      show_gene_labels=True,
                      cmap='RdBu_r',
                      vmin=-2,
                      vmax=2,
                      figsize=figsize,
                      show=False)
        
        # Get the current figure
        fig = plt.gcf()
        plt.tight_layout()
        
        return fig
    else:
        print(f"Error: '{groupby}' column not found in adata.obs")
        return None

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3),  # Set default figure size to 3×3 inches
    'lines.linewidth': 1  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

crit_pval = 0.05
crit_l2fc = 0.5

nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = np.intersect1d(sig_degs_df['names'], all_genes)


fig, ax = plt.subplots(figsize=(3.5,3.5), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=2)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=2)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=2)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=2)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


ax.set_xlim(-2.5, 2.5)  # Adjust these limits as needed
ax.set_ylim(-1, 40)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=6)
         
ax.set_xlabel('Log2FC', fontsize=8)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=8)  # Adjust fontsize as needed
ax.set_title('Severe Treg Versus Other Tregs', fontsize=8)
ax.spines[['right', 'top']].set_visible(False)

fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.png', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.pdf', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.svg', dpi=300, bbox_inches='tight', transparent = False)
plt.show()

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3),  # Set default figure size to 3×3 inches
    'lines.linewidth': 1  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

crit_pval = 0.05
crit_l2fc = 0.5

nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = np.intersect1d(sig_degs_df['names'], all_genes)


fig, ax = plt.subplots(figsize=(3.5,3.5), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=2)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=2)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=2)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=2)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


ax.set_xlim(-2.5, 2.5)  # Adjust these limits as needed
ax.set_ylim(-1, 40)  # Adjust these limits as needed


from adjustText import adjust_text

texts = []
for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    # Create text objects
    texts.append(ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=6))
    
# Adjust text positions to avoid overlaps
adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.3))
         
ax.set_xlabel('Log2FC', fontsize=8)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=8)  # Adjust fontsize as needed
ax.set_title('Severe Treg Versus Other Tregs', fontsize=8)
ax.spines[['right', 'top']].set_visible(False)

fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.png', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.pdf', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.svg', dpi=300, bbox_inches='tight', transparent = False)
plt.show()

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3),  # Set default figure size to 3×3 inches
    'lines.linewidth': 1  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]
marker_intersect = np.intersect1d(sig_degs_df['names'], all_genes)

fig, ax = plt.subplots(figsize=(3.5,3.5), dpi=300)
ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=2)
ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=2)
ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=2)
ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=2)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']

ax.set_xlim(-2.5, 2.5)  # Adjust these limits as needed
ax.set_ylim(-1, 40)  # Adjust these limits as needed

from adjustText import adjust_text
texts = []
for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    # Create text objects with exact coordinates of the point
    texts.append(ax.text(lfc, -np.log10(pval), gene, fontsize=6))
    
# Adjust text positions with improved parameters
adjust_text(texts, 
            arrowprops=dict(arrowstyle='-', color='gray', lw=0.4),
            expand_points=(1.5, 1.5),  # Increase space between points and text
            force_points=(0.5, 0.5),   # Force strength for point-text repulsion
            force_text=(0.5, 0.5),     # Force strength for text-text repulsion
            precision=0.001)           # Higher precision for positioning
         
ax.set_xlabel('Log2FC', fontsize=8)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=8)  # Adjust fontsize as needed
ax.set_title('Severe Treg Versus Other Tregs', fontsize=8)
ax.spines[['right', 'top']].set_visible(False)

fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.png', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.pdf', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig_volvcano.svg', dpi=300, bbox_inches='tight', transparent = False)

plt.show()

In [ ]:
degs_df[degs_df['names'].isin(marker_intersect )]['pvals_adj'].max()

In [ ]:
marker_intersect

In [ ]:
len(sig_genes)

In [ ]:
fig = plot_marker_heatmap(Treg_cells,marker_intersect, groupby='Grade_Svsall', figsize = (4,6))

# Save figure with 300 DPI
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_gene_sig.png', dpi=300, bbox_inches='tight', transparent = True)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic__gene_sig.pdf', dpi=300, bbox_inches='tight', transparent = True)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic__gene_sig.svg', dpi=300, bbox_inches='tight', transparent = True)

In [ ]:
organized_genes = [
    # Canonical_Treg_genes
    "FOXP3", "CTLA4", "IL2RA", "IL2RB", "IL2RG","IKZF1",
    
    # Cytokine_receptors
    "IL1R1", "IL18R1", "IL6ST", "IL2RA", "IL2RB", "IL2RG",
    
    # Interferon_stimulated_genes
    "GBP1", "GBP2", "GBP5", "IRF1", "IRF9",
    
    # Chemokine_receptors
    "CCR4", "CCR6", "CCR8", "CXCR6",
    
    # JAK_STAT
    "JAK1", "STAT1", "STAT3",
    
    # Costimulatory
    "LAG3", "ICOS", "TNFRSF9", "TNFRSF1B",
    
    # Other_transcription_factors
    "IKZF1", "ID2", "FOXO1", "KLF13",
    
    # Cytotoxic Treg
    "PRF1", "GZMA", "GZMB",
    # OxPhos
    "COX6B1", "COX7A2", "COX7C", "NDUFS5", "ATP5F1E", "ATP5MG", "ATP5MC2",
    
    # TCR_components
    "TRBC2",
    
    # MHC_components
    "HLA-DQB1",
]


import matplotlib as mpl
import matplotlib as mpl
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 12,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    #'figure.figsize': (4, 3),  # Set default figure size to 6×4 inches
    'lines.linewidth': 1.2  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['xtick.labelsize'] = 12  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 12  # Colorbar tick label size


# Plot each marker set one at a time and save the figures manually
# Generate the dot plot and return the figure
dotplot = sc.pl.dotplot(
    Treg_cells, 
    var_names=organized_genes, 
    groupby='Grade_Svsall', 
    dendrogram=False, 
    title='',
    return_fig=True,  # Return the figure object for saving
    swap_axes=True,
    figsize=(2.5,10),
    dot_max = 1

)

    
# Save the figure with the specified DPI
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_Treg_cytotoxic.png', dpi=300, bbox_inches='tight')
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_Treg_cytotoxic.pdf', dpi=300, bbox_inches='tight')
dotplot.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/dotplot_Treg_cytotoxic.svg', dpi=300, bbox_inches='tight')
# Show the plot (optional)

plt.show()

In [ ]:
Treg_sig= ['PRF1','GZMA','GZMB','CCL4','IL2RA','IL2RG','IL2RB','FOXP3','GBP2', 'GBP5','IL6ST','STAT3','JAK1','LAG3','IKZF1','CTLA4']


crit_pval = 0.05
crit_l2fc = 0.5

nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = np.intersect1d(sig_degs_df['names'], Treg_sig)

fig, ax = plt.subplots(figsize=(4,4), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


ax.set_xlim(-2, 2)  # Adjust these limits as needed
ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=7)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Severe Treg Versus Other Tregs', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

### To do resend the deg. 

In [ ]:
fig = plot_marker_heatmap(Treg_cells,marker_intersect, groupby='Grade_Svsall', figsize = (4,4))

# Save figure with 300 DPI
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_sig.png', dpi=300, bbox_inches='tight', transparent = True)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_sig.pdf', dpi=300, bbox_inches='tight', transparent = True)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/Treg_cytolytic_sig.svg', dpi=300, bbox_inches='tight', transparent = True)

In [ ]:
sc.tl.rank_genes_groups(Treg_cells, groupby='GVHD_Grade_1', use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(Treg_cells, group=None)
#dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_Treg.csv')

In [ ]:
sc.tl.rank_genes_groups(Treg_cells_100, groupby='ptcy', use_raw=False, method='wilcoxon')

dedf = sc.get.rank_genes_groups_df(Treg_cells_100, group=None)
#dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_Treg_ptcy.csv')

In [ ]:
dedf[dedf['pvals_adj']<0.05]

In [ ]:
dedf = sc.get.rank_genes_groups_df(Treg_cells_100, group='NoP')
dedf[dedf['pvals_adj']<0.05]

In [ ]:
dedf = sc.get.rank_genes_groups_df(Treg_cells_100, group='Ptcy')
dedf[dedf['pvals_adj']<0.05]

In [ ]:
sc.pl.umap(Treg_cells, color=['GVHD_Grade_1'])

In [ ]:
for meta in Treg_cells.obs.columns[0:20]:
    randomize=np.arange(len(Treg_cells.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=Treg_cells.obsm["X_umap"][:,0][randomize],y=Treg_cells.obsm["X_umap"][:,1][randomize], hue=Treg_cells.obs[meta][randomize],s=10).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sc.pp.neighbors(Treg_cells, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)

In [ ]:
sc.tl.leiden(Treg_cells,resolution=0.25)
sc.pl.umap(Treg_cells, color=['GVHD_Grade_1','FOXP3','IL2RA','leiden'])

In [ ]:
sc.pl.violin(Treg_cells, ['FOXP3','IL2RA','CTLA4','ENTPD1','IRF1','IL2RG','IL2RB'], groupby='GVHD_Grade_1')

In [ ]:
sc.pl.violin(Treg_cells, ['FOXP3','IL2RA','CTLA4','IL2RG','IL2RB','IL12A'], groupby='GVHD_Grade_1')

In [ ]:
sc.pl.violin(Treg_cells, ['IL12A','AREG','MAF','IKZF2','IL10','PDCD1','IKZF2','NT5E'], groupby='GVHD_Grade_1')

In [ ]:
sc.pl.violin(Treg_cells, ['PDCD1','IKZF2','NT5E','NRP1'], groupby='GVHD_Grade_1')

In [ ]:
sc.pl.violin(Treg_cells, ['IL12A','AREG','MAF','IKZF2','IL10','FOXP3','CD3E','CD3D','IL2RG','CTLA4'], groupby='leiden')

In [ ]:
sc.tl.rank_genes_groups(Treg_cells, groupby='leiden', use_raw=False, method='wilcoxon')

dedf = sc.get.rank_genes_groups_df(Treg_cells, group=None)
dedf.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/dedf_Treg_leiden.csv')

In [ ]:
dedf = sc.get.rank_genes_groups_df(Treg_cells, group='0')
# highly expressed in cluster 0
# Mitochondrial genes
list(dedf[(dedf['pvals_adj']<0.05) & (dedf['scores']>0)]['names'])

In [ ]:
# highly expressed in cluster 1
# Treg functional gene
list(dedf[(dedf['pvals_adj']<0.05) & (dedf['scores']<0)]['names'])

In [ ]:
Treg_cells

In [ ]:
sc.pl.umap(Treg_cells, color=['GVHD_Grade_1','FOXP3','IL2RA','leiden','ptcy','PatientID_Grade','LPorIE','Tissue_Location'])

In [ ]:
sns.set_style('white')

font_list = []
fpaths = matplotlib.font_manager.findSystemFonts()
for i in fpaths:
    try:
        f = matplotlib.font_manager.get_font(i)
        font_list.append(f.family_name)
    except RuntimeError:
        pass

font_list = set(font_list)
plot_font = 'Helvetica' if 'Helvetica' in font_list else 'FreeSans'

rcParams['font.family'] = plot_font
rcParams.update({'font.size': 15})
rcParams.update({'figure.dpi': 100})
rcParams.update({'figure.figsize': (4, 4)})
rcParams.update({'savefig.dpi': 100})

import warnings
warnings.filterwarnings('ignore')

In [ ]:
font_list

In [ ]:
from copy import deepcopy
obs_1 = 'GVHD_Grade_1'
obs_2 = 'leiden'

n_categories = {x : len(Treg_cells.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = Treg_cells.obs[[obs_2, obs_1]].values

obs2_clusters = Treg_cells.obs[obs_2].cat.categories.tolist()
obs1_clusters = Treg_cells.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters

df_obs2.T.plot(kind="bar", stacked=True, color=Treg_cells.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=Treg_cells.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
from copy import deepcopy
obs_1 = 'ptcy'
obs_2 = 'leiden'

n_categories = {x : len(Treg_cells.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = Treg_cells.obs[[obs_2, obs_1]].values

obs2_clusters = Treg_cells.obs[obs_2].cat.categories.tolist()
obs1_clusters = Treg_cells.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters

df_obs2.T.plot(kind="bar", stacked=True, color=Treg_cells.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=Treg_cells.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
from copy import deepcopy
obs_1 = 'PatientID_Grade'
obs_2 = 'leiden'

n_categories = {x : len(Treg_cells.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = Treg_cells.obs[[obs_2, obs_1]].values

obs2_clusters = Treg_cells.obs[obs_2].cat.categories.tolist()
obs1_clusters = Treg_cells.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
#df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True, color=Treg_cells.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=Treg_cells.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
from copy import deepcopy
obs_1 = 'LPorIE'
obs_2 = 'leiden'

n_categories = {x : len(Treg_cells.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = Treg_cells.obs[[obs_2, obs_1]].values

obs2_clusters = Treg_cells.obs[obs_2].cat.categories.tolist()
obs1_clusters = Treg_cells.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
#df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True, color=Treg_cells.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=Treg_cells.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
from copy import deepcopy
obs_1 = 'Tissue_Location'
obs_2 = 'leiden'

n_categories = {x : len(Treg_cells.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = Treg_cells.obs[[obs_2, obs_1]].values

obs2_clusters = Treg_cells.obs[obs_2].cat.categories.tolist()
obs1_clusters = Treg_cells.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
#df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True, color=Treg_cells.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=Treg_cells.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
Treg_cells[Treg_cells.obs['PatientID_Grade'] == 'C66_No'] 

In [ ]:
tcell_ann_concat.obs["ptcy"]=tcell_ann_concat.obs["ptcy"].astype("category")

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['ptcy'])

In [ ]:
from copy import deepcopy
obs_1 = 'ptcy'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['PatientID_Grade_Ptcy'].unique()

In [ ]:
#ptcy_sorter = ['ND001_ND_No_Ptcy','ND003_ND_No_Ptcy','C155_No_Ptcy','C157_Mild_No_Ptcy','R085_Mild_No_Ptcy','C179_Mild_No_Ptcy','C66_Mild_No_Ptcy','C101_Mild_Ptcy','C162_Mild_Ptcy','C160_Severe_Ptcy','C159_Severe_No_Ptcy','C152_Severe_No_Ptcy','C068_Severe_No_Ptcy','C98_Severe_No_Ptcy','C66_Severe_No_Ptcy']

In [ ]:
ptcy_sorter = ['C155_No_Ptcy','C101_Mild_Ptcy','C162_Mild_Ptcy','C160_Severe_Ptcy','ND001_ND_NoP','ND003_ND_NoP','R085_No_NoP','C157_Mild_NoP','C179_Mild_NoP','C66_Mild_NoP','C159_Severe_NoP','C152_Severe_NoP','C068_Severe_NoP','C98_Severe_NoP','C66_Severe_NoP']

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['PatientID_Grade_Ptcy'])

In [ ]:
tcell_ann_concat.obs.columns

In [ ]:
tcell_ann_concat_100 = tcell_ann_concat[tcell_ann_concat.obs['Days_post-Transplant']<=100]
tcell_ann_concat_100

In [ ]:
from copy import deepcopy
obs_1 = 'PatientID_Grade_Ptcy'
obs_2 = 'leiden_0.5'
num = 1
#tcell_ann_concat, tcell_ann_concat_Upper,tcell_ann_concat_Lower,tcell_ann_concat_Upper_LP,tcell_ann_concat_Upper_IE,tcell_ann_concat_Lower_LP,tcell_ann_concat_Lower_IE,
for adata in [tcell_ann_concat_100]:
    n_categories = {x : len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])
        
    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]
            
    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]
            
    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters
    df_obs2 = df_obs2.reindex(columns=ptcy_sorter)
    df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()
    #plt.title(adata)
    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True,color=tcell_ann_concat.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()
    #plt.title(adata)

    print(num)
    num = num +1

    for i in adata.obs['leiden_0.5'].unique():
        substring = 'Ptcy'
        column_mask = df_obs2.columns.str.contains(substring)
        # Use the mask to select the desired columns
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]
        substring = 'NoP'
        column_mask = df_obs2.columns.str.contains(substring)
        # Use the mask to select the desired columns
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]
        
        t_stat, p_value = stats.ttest_ind(severe, mild)
        print(i)
        print(p_value)
        
        if p_value<=0.05:
            plt.boxplot([mild, severe], labels=['NoP', 'Ptcy'])

        # Add labels and title
            plt.xlabel('Ptcy')
            plt.ylabel('Cell type Proportion')
            plt.title(i)
            plt.show()

In [ ]:
from copy import deepcopy
obs_1 = 'LPorIE'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
sc.pl.umap(tcell_ann_concat, color='Tissue_Location')

In [ ]:
obs_1 = 'Tissue_Location'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat

In [ ]:
sc.pl.umap(tcell_ann_concat, color='GVHD_Grade_1')

In [ ]:
from copy import deepcopy
obs_1 = 'GVHD_Grade_1'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['PatientID_Time_Tissue'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['Days_post-Transplant'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)

In [ ]:
sc.pl.umap(tcell_ann_concat, color='PatientID_Time_Tissue')

In [ ]:
list(tcell_ann_concat.obs['PatientID_Time_Tissue'].unique())

In [ ]:
sorter = ['C155_27.0_Upper GI','C101_32.0_Lower GI','C162_33.0_Upper GI',
 'C162_33.0_Lower GI','C157_23.0_Lower GI',
 'C157_23.0_Upper GI','R085_76.0_Upper GI','C160_28.0_Lower GI',
 'C160_196.0_Upper GI','C159_36.0_Upper GI',
 'C159_36.0_Lower GI','C152_164.0_Upper GI',
 'C152_164.0_Lower GI','C068_140.0_Upper GI','C98_119.0_Upper GI',
 'C98_119.0_Lower GI','C66_375.0_Upper GI',
 'C66_444.0_Upper GI',
 'C66_444.0_Lower GI','C179_45.0_Lower GI',
 'C179_45.0_Upper GI','ND001_nan_Upper GI',
 'ND001_nan_Lower GI','ND003_nan_Lower GI',
 'ND003_nan_Upper GI']

In [ ]:
obs_1 = 'PatientID_Time_Tissue'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]



df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.index = obs2_clusters
df_obs2.columns = obs1_clusters
df_obs2 = df_obs2.reindex(columns=sorter)

df_obs2.T.plot(kind="bar", stacked=True, color=tcell_ann_concat.uns[obs_2 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')


df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.index = obs1_clusters
df_obs1.columns = obs2_clusters

df_obs1.T.plot(kind="bar", stacked=True,color=tcell_ann_concat.uns[obs_1 + '_colors'])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['Tissue_LPorIE'] = tcell_ann_concat.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat.obs['LPorIE'].astype(str)

sc.pl.umap(tcell_ann_concat, color='Tissue_LPorIE')

In [ ]:
from copy import deepcopy
obs_1 = 'Tissue_LPorIE'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['Tissue_LPorIE_Grade'] = tcell_ann_concat.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat.obs['LPorIE'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)

sc.pl.umap(tcell_ann_concat, color=['Tissue_LPorIE_Grade'])

In [ ]:
from copy import deepcopy
obs_1 = 'Tissue_LPorIE_Grade'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['Patient_Tissue_LPorIE_Grade'] = tcell_ann_concat.obs['PatientID'].astype(str)+'_'+tcell_ann_concat.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat.obs['LPorIE'].astype(str)+'_'+tcell_ann_concat.obs['GVHD_Grade_1'].astype(str)

sc.pl.umap(tcell_ann_concat, color=['Patient_Tissue_LPorIE_Grade'])

In [ ]:
tcell_ann_loc = '/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_concat_Tcell_only_annotated.h5ad'
tcell_ann_concat.write(tcell_ann_loc)

In [ ]:
tcell_ann_concat = anndata.read_h5ad('/burg/iicd/users/ls3456/GVHD/adata_file/tcell_ann_concat_Tcell_only_annotated.h5ad')

In [ ]:
tcell_ann_concat

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'])

In [ ]:
from copy import deepcopy
obs_1 = 'Patient_Tissue_LPorIE_Grade'
obs_2 = 'leiden_0.5'

n_categories = {x : len(tcell_ann_concat.obs[x].cat.categories) for x in [obs_1, obs_2]}
df = tcell_ann_concat.obs[[obs_2, obs_1]].values

obs2_clusters = tcell_ann_concat.obs[obs_2].cat.categories.tolist()
obs1_clusters = tcell_ann_concat.obs[obs_1].cat.categories.tolist()

obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                   for k in obs1_clusters}
obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                   for k in obs2_clusters}

for b, l in df:
  obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
  obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1

obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))

for i, k in enumerate(obs2_clusters):
  obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
for i, k in enumerate(obs1_clusters):
  obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

sums = np.sum(obs2_to_obs1_array, 0)
for col in range(np.size(obs2_to_obs1_array, 1)):
  for row in range(np.size(obs2_to_obs1_array, 0)):
    obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

sums2 = np.sum(obs1_to_obs2_array, 0)
for col in range(np.size(obs1_to_obs2_array, 1)):
  for row in range(np.size(obs1_to_obs2_array, 0)):
    obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

df_obs2 = pd.DataFrame(obs2_to_obs1_array)
df_obs2.head()

x = obs1_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_2]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_2 + '_colors'][0]
  for j in range(i):
    bot += df_obs2.iloc[j]
    color = tcell_ann_concat.uns[obs_2 + '_colors'][i]
  plt.bar(x, df_obs2.iloc[i], bottom=bot, color=color, label=obs2_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

df_obs1 = pd.DataFrame(obs1_to_obs2_array)
df_obs1.head()

x = obs2_clusters
plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
plt.grid(False)

# stack bars
for i in range(n_categories[obs_1]):
  bot = 0.0
  color = tcell_ann_concat.uns[obs_1 + '_colors'][0]
  for j in range(i):
    bot += df_obs1.iloc[j]
    color = tcell_ann_concat.uns[obs_1 + '_colors'][i]
  plt.bar(x, df_obs1.iloc[i], bottom=bot, color=color,
          label=obs1_clusters[i])
plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')

In [ ]:
tcell_ann_concat.obs['leiden_0.5']

# CD4+ T cels

In [ ]:
CD4_cell_noTreg = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'].isin(['CD4+ Central Memory','Effector CD4+ T-cells'])]

In [ ]:
sc.pp.highly_variable_genes(CD4_cell_noTreg, n_top_genes=5000, subset=False, layer="counts",batch_key ='PatientID' )

In [ ]:
CD4_cell_noTreg.var['highly_variable']

In [ ]:
CD4_cell_noTreg = CD4_cell_noTreg[:,CD4_cell_noTreg.var['highly_variable']== True]
CD4_cell_noTreg

In [ ]:
sc.pp.neighbors(CD4_cell_noTreg)

In [ ]:
sc.tl.umap(CD4_cell_noTreg)

In [ ]:
sc.pl.umap(CD4_cell_noTreg, color=['leiden',"CD3D","CD3E","CD3G", 'CD4', 'CD8A','SampleID', 'PatientID'])

In [ ]:
sc.pp.neighbors(CD4_cell_noTreg, n_neighbors=30, use_rep="X", n_pcs= 25)
sc.tl.umap(CD4_cell_noTreg)
sc.pl.umap(CD4_cell_noTreg, color=['leiden',"CD3D","CD3E","CD3G", 'CD4', 'CD8A','SampleID', 'PatientID'])

In [ ]:
sc.pp.neighbors(CD4_cell_noTreg, n_neighbors=30, use_rep="X_scVI", n_pcs= 25)

In [ ]:
sc.tl.umap(CD4_cell_noTreg,)

In [ ]:
sc.pl.umap(CD4_cell_noTreg, color=['leiden',"CD3D","CD3E","CD3G", 'CD4', 'CD8A'])

In [ ]:
sc.tl.leiden(CD4_cell_noTreg,resolution=0.5)#resolution=0.1,

In [ ]:
#th1
sc.pl.umap(CD4_cell_noTreg, color=['TBX21','IFNG','STAT4','STAT1','TNF','CXCR3','IL2','IL12RB1','IL12RB2','IL18R1','LTA','LTB'], ncols=3)

#th17
sc.pl.umap(CD4_cell_noTreg, color=['RORC','IL17A','IL17F','AHR','IKZF3','STAT3','IL23R','IKZF1','CCR6','IL21','IL7R'], ncols=4)

# TH2
sc.pl.umap(CD4_cell_noTreg, color=['GATA3', 'IL13','IL5' ])

# T cell activation marker
sc.pl.umap(CD4_cell_noTreg, color=['IL2RA','CD69','TNFRSF4','CD40LG','LTB','ICOS'], ncols=2)

# Naive Resting
sc.pl.umap(CD4_cell_noTreg, color=['CCR7', 'SELL'])

In [ ]:
# Volcano plot

sc.tl.rank_genes_groups(CD4_cell_noTreg, groupby='leiden_0.5', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(CD4_cell_noTreg, group='Effector CD4+ T-cells')
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]


crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(6,6), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=2)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Effector CD4+ T-cells vs CD4+ Central Memory', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()


In [ ]:
CD4_cell_noTreg

# Homestatic conventional vs cytotoxic 

In [ ]:
Unconventional_CD8 = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'].isin(['CD8+ Cytotoxic Unconventional T cells','CD8+ Homeostatic Unconventional T cells '])]

In [ ]:
tcell_ann_concat.obs['leiden_0.5'].unique()

In [ ]:
from copy import deepcopy
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3),  # Set default figure size to 3×3 inches
    'lines.linewidth': 1  # Set default line width to 1
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

sc.tl.rank_genes_groups(Unconventional_CD8, groupby='leiden_0.5', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(Unconventional_CD8, group='CD8+ Cytotoxic Unconventional T cells')
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]
degs_df.to_csv('/Users/lingting/Documents/GVHD_project/HomeostaticvsT.csv')


crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(6,6), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=2)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Effector CD4+ T-cells vs CD4+ Central Memory', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
# Define the upregulated genes by category
upregulated = {
    "TCR Signaling Components": ["CD3E", "CD3D", "CD247", "PTPRC", "LCP1", "LCP2", "FYB1"],
    
    "Co-stimulation/Coreceptors": ["CD27", "CD28", "ICOS", "CD4", "CD5", "CD6", "CD7", "CD2", 
                                   "CD40LG", "TNFRSF4", "TNFRSF18", "TNFRSF25"],
    
    "Activation Markers and TFs": ["CD69", "JUN", "FOS", "JUNB", "RORC", "EOMES", "ID2", "IKZF3"],
    
    "NF-kB Pathway": ["NFKBIA", "NFKBIZ", "TNFAIP3"],
    
    "MAPK Pathway": ["DUSP1", "DUSP2"],
    
    "Pro-inflammatory Cytokines": ["IFNG", "TNF", "LTB", "IL32", "MIF"],
    
    "Chemokines": ["CCL3", "CCL4", "CCL5", "CCL20", "XCL1"],
    
    "Cytokine/Chemokine Receptors": ["IL2RG", "CCR6"],
    
    "Interferon Response": ["IFNG", "STAT1", "IRF1", "IFITM1", "IFITM2", "GBP1", "GBP4", 
                           "GBP5", "PSME1", "PSME2"],
    
    "Effector Functions (Cytotoxicity)": ["GZMA", "GZMB", "GZMK", "GNLY", "NKG7", "FCER1G", "TYROBP"],
    
    "Antigen Presentation & Processing": ["HLA-A", "HLA-B", "HLA-C", "HLA-E", "B2M", "HLA-DRA", 
                                         "HLA-DRB1", "HLA-DPB1", "CD74", "PSME1", "PSME2", 
                                         "CTSB", "CTSH"],
    
    "Integrins/Adhesion": ["ITGB2", "ITM2A", "ITM2B", "CD81", "CD82"],
    
    "Cytoskeletal remodeling and motility": ["ACTB", "ACTG1", "PFN1", "CFL1", "CORO1A", 
                                            "ARPC1B", "ARPC2", "ARPC3", "ACTR2", "EVL", 
                                            "PLEK", "TUBA1B", "STMN1", "MYL6", "MYL12A", 
                                            "MYL12B", "MYH9", "RAC2", "RHOA", "TMSB4X", 
                                            "TMSB10", "VIM", "TPM3", "TPM4", "MACF1"],
    
    "Glycolysis": ["SLC2A3", "ATP5F1E"],
    
    "Cell Cycle/Proliferation": ["MKI67", "RRM2", "TYMS", "NUSAP1", "LMNB1"],
    
    "Stress Response/Chaperones": ["HSPA8", "HSP90AA1", "HSP90AB1", "HSPA5", "CALR"],
    
    "Co-inhibitory Receptors": ["CTLA4", "PDCD1", "LAG3"]
}

# Define the downregulated genes by category
downregulated = {
    "OXPHOS": ["NDUFV1", "NDUFS5", "NDUFC2", "NDUFA1", "COX7A2L", "COX14", 
              "ATP5MPL", "ATP5MD", "ATP5ME", "ATP6V1G1", "ATP8B4", "ATP8A1",
              "TOMM20", "TOMM7", "MICOS13", "CHCHD10", "HSPE1", "MPC2", 
              "PET100", "PRDX5"],
    
    "Inhibitory receptors": ["CD96", "CD160", "KLRC2", "KLRC3", "KLRD1", 
                            "KIR2DL4", "CD101", "SIRPG"],
    
    "Adhesion": ["ITGB7", "ITGA1"],
    
    "TFs": ["IKZF2", "NFIL3"],
    
    "Glucocorticoid receptor": ["NR3C1"],
    
    "Apoptosis": ["BCL2"],
    
    "STAT pathway": ["STAT4"],
    
    "Negative regulators of TCR signaling": ["CBLB", "PTPN22", "PTPN4", "SOCS1", 
                                            "PIK3IP1", "SPRY1", "UBASH3B"],
    
    "Lipid and cholesterol biosynthesis": ["CYP51A1", "MSMO1", "IDI1", "FDFT1", 
                                          "INSIG1", "HSD17B11", "ECH1", "ECI2", 
                                          "PNPLA2", "PLPP1", "MGST3", "DHRS7", 
                                          "LDLRAD4"]
}

# Create flat lists for all genes
all_upregulated_genes = []
for category, genes in upregulated.items():
    all_upregulated_genes.extend(genes)

all_downregulated_genes = []
for category, genes in downregulated.items():
    all_downregulated_genes.extend(genes)

# Remove any duplicates
all_upregulated_genes = list(set(all_upregulated_genes))
all_downregulated_genes = list(set(all_downregulated_genes))

# Combined list with regulation status
all_genes_with_status = []
for gene in all_upregulated_genes:
    all_genes_with_status.append((gene, "upregulated"))
for gene in all_downregulated_genes:
    all_genes_with_status.append((gene, "downregulated"))

# Print summary statistics
print(f"Total upregulated genes: {len(all_upregulated_genes)}")
print(f"Total downregulated genes: {len(all_downregulated_genes)}")
print(f"Total genes: {len(all_genes_with_status)}")

In [ ]:
import matplotlib as mpl
# Update your existing rcParams
# Update your existing rcParams
mpl.rcParams.update({
    'font.size': 8,
    'svg.fonttype': 'none',
    'font.family': 'Arial',
    'figure.figsize': (3, 3)
})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size

crit_pval = 0.001
crit_l2fc = 1

nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = np.intersect1d(sig_degs_df['names'], all_genes_with_status)


fig, ax = plt.subplots(figsize=(3.5,3.5), dpi=300)
ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=0.5)
ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=0.5)
ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=0.5)
ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=0.5)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']

ax.set_xlim(-7, 7)  # Adjust these limits as needed
ax.set_ylim(-1, 300)  # Adjust these limits as needed

from adjustText import adjust_text
texts = []
for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    # Create text objects with exact coordinates of the point
    texts.append(ax.text(lfc, -np.log10(pval), gene, fontsize=3.2))
    
# Adjust text positions with improved parameters
adjust_text(texts, 
            arrowprops=dict(arrowstyle='-', color='gray', lw=0.4),
            expand_points=(1.5, 1.5),  # Increase space between points and text
            force_points=(0.5, 0.5),   # Force strength for point-text repulsion
            force_text=(0.5, 0.5),     # Force strength for text-text repulsion
            precision=0.001)           # Higher precision for positioning
         
ax.set_xlabel('Log2FC', fontsize=8)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=8)  # Adjust fontsize as needed
ax.set_title('CD8+ Unconventional cytotoxic Versus \n CD8+ Homeostatic Unconventionl T cells', fontsize=8)
ax.spines[['right', 'top']].set_visible(False)

fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/unconventional_gene_sig_volvcano.png', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/unconventional_gene_sig_volvcano.pdf', dpi=300, bbox_inches='tight', transparent = False)
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/single_cell_figures/unconventional_gene_sig_volvcano.svg', dpi=300, bbox_inches='tight', transparent = False)
plt.show()